In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache
from matplotlib import cm
import operator
import matplotlib.ticker as ticker
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

sns.set_context('poster', font_scale = 1.5)
sns.set_style('white')
%matplotlib inline

import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42

In [ ]:
path = r'/Users/jenniferwh/Dropbox (Allen Institute)/Mesoscale Connectome Papers in Progress/2018 Plaque'

In [ ]:
savepath = os.path.join(path, 'figures')

In [ ]:
dat = pd.read_csv(os.path.join(path, 'unionizes', 'plaque_densities_7-13.csv'))

In [ ]:
dat.head()

In [ ]:
dat_count = pd.read_csv(os.path.join(path, 'plaque counts', 'plaque_counts_7-13.csv'))

In [ ]:
dat_count = dat_count.merge(dat[['image_series_id', 'structure_acronym', 'structure_volume']], 
                on=['image_series_id', 'structure_acronym'])

In [ ]:
dat_count['count_per_mm_3'] = dat_count['sum_plaques']/dat_count['structure_volume']

In [ ]:
dat_count.head()

In [ ]:
meta = pd.read_csv(os.path.join(path, 'filtered_plaque_dataset_07_20_2018.csv'))

In [ ]:
dataset = meta[meta['Plaque map call (R hemisphere)'] == 'Pass']
print(len(dataset))

In [ ]:
mcc = MouseConnectivityCache(manifest_file = '../connectivity/mouse_connectivity_manifest.json')
st = mcc.get_structure_tree()
summary_structures = st.get_structures_by_set_id([167587189])
summary_structure_ids = [item['id'] for item in summary_structures]
ss_acronyms = [item['acronym'] for item in summary_structures]
ont_order = [item['graph_order'] for item in summary_structures]
ont_order_dict = dict(zip(ss_acronyms, ont_order))
print(len(summary_structure_ids))


In [ ]:
coarse_structures = st.get_structures_by_set_id([687527670])
coarse_structure_ids = [item['id'] for item in coarse_structures]
coarse_structure_acronyms = [item['acronym'] for item in coarse_structures]
ft = st.get_structures_by_acronym(['fiber tracts'])
coarse_structure_ids.append(ft[0]['id'])
coarse_structure_acronyms.append(ft[0]['acronym'])
ont_order = [item['graph_order'] for item in coarse_structures]
ont_order.append(ft[0]['graph_order'])
oo_dict_coarse = dict(zip(coarse_structure_acronyms, ont_order))
print(len(coarse_structure_ids))

In [ ]:
mcc = MouseConnectivityCache(manifest_file = '../connectivity/mouse_connectivity_manifest.json')
structure_tree = mcc.get_structure_tree()
ia_map = structure_tree.get_id_acronym_map()
ai_map = {value:key for key, value in ia_map.iteritems()}

In [ ]:
structures = summary_structure_ids + coarse_structure_ids + [997]

In [ ]:
dataset_ids = dat['image_series_id'].unique()
print(len(dataset_ids))

In [ ]:
isids = [int(value) for value in dataset['Link to image series']]

In [ ]:
dataset['image_series_id'] = isids

In [ ]:
dataset = dataset[dataset['image_series_id'].isin(dataset_ids)]
print(len(dataset))

In [ ]:
dataset.head()

In [ ]:
dataset.groupby(['Mouse Line', 'age group']).count()

In [ ]:
drop_structures = [acronym for acronym in dat['structure_acronym'] if acronym not in ia_map]
print(len(drop_structures))

In [ ]:
print(len(dat))
dat = dat[~dat['structure_acronym'].isin(drop_structures)]
print(len(dat))

In [ ]:
dat['structure_id'] = [ia_map[structure_acronym] for structure_acronym in dat['structure_acronym']]

In [ ]:
drop_structures = [acronym for acronym in dat_count['structure_acronym'] if acronym not in ia_map]
print(len(drop_structures))

In [ ]:
print(len(dat_count))
dat_count = dat_count[~dat_count['structure_acronym'].isin(drop_structures)]
print(len(dat_count))
dat_count['structure_id'] = [ia_map[structure_acronym] for structure_acronym in dat_count['structure_acronym']]

In [ ]:
def get_structure_data(age, structure, q_type):
    if q_type == 'density':
        data = dat
        data = data.rename(columns = {'plaque_density': 'plaque'})
    elif q_type == 'count':
        data = dat_count
        data = data.rename(columns = {'count_per_mm_3': 'plaque'})    
    structure_id = ia_map[structure]
    groups = []
    plaques = []
    isids_carrythrough = []
    for group in dataset[dataset['age group'] == age]['Mouse Line'].unique():
        isids = dataset[(dataset['Mouse Line'] == group) & (dataset['age group'] == age)]['image_series_id'].values
        for isid in isids:
            groups.append(group)
            if len(data[(data['structure_id'] == structure_id) & (
                data['image_series_id'] == isid)]['plaque']) > 0:
                plaques.append(data[(data['structure_id'] == structure_id) & 
                                         (data['image_series_id'] == isid)]
                                     ['plaque'].values[0])
            else:
                print(isid)
                print(ai_map[structure_id])
                plaques.append(0)
                    
            isids_carrythrough.append(isid)
    structuredat = pd.DataFrame({'age': age, 'structure': structure, 
                                 'group': groups, 'plaques': plaques,
                                'image_series_id': isids_carrythrough})
    return structuredat

def get_structure_data_tau(age, structure, q_type):
    if q_type == 'density':
        data = dat
        data = data.rename(columns = {'plaque_density': 'plaque'})
    elif q_type == 'count':
        data = dat_count
        data = data.rename(columns = {'count_per_mm_3': 'plaque'})    
    structure_id = ia_map[structure]
    groups = []
    plaques = []
    isids_carrythrough = []
    for group in tau_dataset[tau_dataset['age group'] == age]['Mouse Line'].unique():
        isids = tau_dataset[(tau_dataset['Mouse Line'] == group) & 
                            (tau_dataset['age group'] == age)]['image_series_id'].values
        for isid in isids:
            groups.append(group)
            if len(data[(data['structure_id'] == structure_id) & (
                data['image_series_id'] == isid)]['plaque']) > 0:
                plaques.append(data[(data['structure_id'] == structure_id) & 
                                         (data['image_series_id'] == isid)]
                                     ['plaque'].values[0])
            else:
                print(isid)
                print(ai_map[structure_id])
                plaques.append(0)
                    
            isids_carrythrough.append(isid)
    structuredat = pd.DataFrame({'age': age, 'structure': structure, 
                                 'group': groups, 'plaques': plaques,
                                'image_series_id': isids_carrythrough})
    return structuredat

In [ ]:
def get_mean_structure_data(age, structure):
    structure_id = ia_map[structure]
    groups = []
    n = []
    means = []
    stds = []
    for group in dataset[dataset['age group'] == age]['Mouse Line'].unique():
        groups.append(group)
        isids = dataset[(dataset['Mouse Line'] == group) & (dataset['age group'] == age)]['image_series_id'].values
        n.append(len(isids))
        means.append(np.mean(dat[(dat['structure_id'] == structure_id) & 
                             (dat['image_series_id'].isin(isids))]
                        ['plaque_density']))
        stds.append(np.std(dat[(dat['structure_id'] == structure_id) & 
                             (dat['image_series_id'].isin(isids))]
                        ['plaque_density']))
    structuredat = pd.DataFrame({'age': age, 'group': groups, 'n':n, 
                                 'structure': structure, 'mean':means, 'standard deviation':stds})
    return structuredat

def get_mean_structure_count(age, structure):
    structure_id = ia_map[structure]
    groups = []
    n = []
    means = []
    stds = []
    for group in dataset[dataset['age group'] == age]['Mouse Line'].unique():
        groups.append(group)
        isids = dataset[(dataset['Mouse Line'] == group) & (dataset['age group'] == age)]['image_series_id'].values
        n.append(len(isids))
        means.append(np.mean(dat_count[(dat_count['structure_id'] == structure_id) & 
                             (dat_count['image_series_id'].isin(isids))]
                        ['count_per_mm_3']))
        stds.append(np.std(dat_count[(dat_count['structure_id'] == structure_id) & 
                             (dat_count['image_series_id'].isin(isids))]
                        ['count_per_mm_3']))
    structuredat = pd.DataFrame({'age': age, 'group': groups, 'n':n, 
                                 'structure': structure, 'mean plaques per mm^3':means, 
                                 'standard deviation plaques per mm^3':stds})
    return structuredat

In [ ]:
def get_relative_structure_data(age, structure):
    structure_id = ia_map[structure]
    groups = []
    plaque_density = []
    for group in dataset[dataset['age group'] == age]['Mouse Line'].unique():
        isids = dataset[(dataset['Mouse Line'] == group) & (dataset['age group'] == age)]['image_series_id'].values
        for isid in isids:
            groups.append(group)
            structure_projection_volume = dat[(dat['structure_id'] == structure_id) & 
                                         (dat['image_series_id'] == isid)]['plaque_density'].values[0]
            total_volume = dat[(dat['structure_id'] == 997) & # relative to root
                               (dat['image_series_id'] == isid)]['plaque_density'].values[0]
            plaque_density.append(structure_projection_volume/total_volume)
    structuredat = pd.DataFrame({'age': age, 'structure': structure, 
                                 'group': groups, 'plaque density': plaque_density})
    return structuredat

In [ ]:
pltdat = get_mean_structure_data('12 mo', 'HPF')
pd.concat([pltdat, get_mean_structure_data('12 mo', 'Isocortex')])

## Means are slow - not calculating every time

In [ ]:
meandat = pd.DataFrame(columns={'age', 'group', 'mean', 'n', 'standard deviation', 'structure'})
for age in dataset['age group'].unique():
    for structure in structures:
        meandat = pd.concat([meandat, get_mean_structure_data(age, ai_map[structure])])

In [ ]:
meandat.to_csv(os.path.join(path, 'plaque density mean SD by mouse line and structure.csv'))

In [ ]:
meancount = pd.DataFrame(columns={'age', 'group', 'mean', 'n', 'standard deviation', 'structure'})
for age in dataset['age group'].unique():
    for structure in structures:
        meancount = pd.concat([meandat, get_mean_structure_count(age, ai_map[structure])])
meancount.to_csv(os.path.join(path, 'plaque count mean SD by mouse line and structure.csv'))

In [ ]:
tau_dataset = dataset[dataset['Mouse Line'].isin(['APP+/Tau+', 'APP+/Tau-'])]

In [ ]:
dataset = dataset[~dataset['Mouse Line'].isin(['APP+/Tau+', 'APP+/Tau-'])]

In [ ]:
dataset['Mouse Line'].unique()

In [ ]:
pltdat = get_structure_data('12 mo', 'root', 'density')

In [ ]:
mdf = pd.melt(pltdat, id_vars = ['age', 'structure', 'group', 'image_series_id'], value_name = 'plaque density')

In [ ]:
fig, ax = plt.subplots(1, figsize = (7, 5))
sns.boxplot('group', 'plaque density', data=mdf, ax=ax, color = "white")
sns.swarmplot('group', 'plaque density', data=mdf, ax=ax, s=7, alpha = 0.8)
plt.title('Brain-wide Plaque Density at 12 Months')

In [ ]:
mdf[mdf['plaque density'].isnull()]

In [ ]:
mdf.head()

In [ ]:
pltdat = get_structure_data('12 mo', 'root', 'count')
mdf = pd.melt(pltdat, id_vars = ['age', 'structure', 'group', 'image_series_id'], 
              value_name = 'plaques per mm^3')
fig, ax = plt.subplots(1, figsize = (10, 5))
sns.boxplot('group', 'plaques per mm^3', data=mdf, ax=ax, color = "white")
sns.swarmplot('group', 'plaques per mm^3', data=mdf, ax=ax, s=7, alpha = 0.8)
plt.title('Brain-Wide Plaque Count at 12 Months')

In [ ]:
structure = 'root'
structure_id = ia_map[structure]
ages = []
groups = []
plaque_density = []
isids_carrythrough = []
for age in dataset['age group'].unique():
    for group in dataset[dataset['age group'] == age]['Mouse Line'].unique():
        isids = dataset[(dataset['Mouse Line'] == group) & (dataset['age group'] == age)]['image_series_id'].values
        for isid in isids:
            ages.append(age)
            groups.append(group)
            plaque_density.append(dat[(dat['structure_id'] == structure_id) & 
                                         (dat['image_series_id'] == isid)]
                                     ['plaque_density'].values[0])
            isids_carrythrough.append(isid)
density_dat = pd.DataFrame({'age': ages, 'structure': structure, 
                             'group': groups, 'plaque density': plaque_density,
                             'image_series_id': isids_carrythrough})

In [ ]:
fig, ax = plt.subplots(1, figsize = (7, 5))
g = sns.swarmplot('age', 'plaque density', hue = 'group', data=density_dat, ax=ax, s=7, alpha = 0.8)
g.set_ylim([0, 0.02])
plt.title('Whole Brain Plaque Density by Age')

In [ ]:
dat_count.head()

In [ ]:
structure = 'root'
structure_id = ia_map[structure]
ages = []
groups = []
plaque_count = []
isids_carrythrough = []
for age in dataset['age group'].unique():
    for group in dataset[dataset['age group'] == age]['Mouse Line'].unique():
        isids = dataset[(dataset['Mouse Line'] == group) & (dataset['age group'] == age)]['image_series_id'].values
        for isid in isids:
            ages.append(age)
            groups.append(group)
            plaque_count.append(dat_count[(dat_count['structure_id'] == structure_id) & 
                                         (dat_count['image_series_id'] == isid)]
                                     ['count_per_mm_3'].values[0])
            isids_carrythrough.append(isid)
count_dat = pd.DataFrame({'age': ages, 'structure': structure, 
                             'group': groups, 'plaque count': plaque_count,
                             'image_series_id': isids_carrythrough})
fig, ax = plt.subplots(1, figsize = (7, 5))
sns.swarmplot('age', 'plaque count', hue = 'group', data=count_dat, ax=ax, s=7, alpha = 0.8)
plt.title('Whole Brain Plaque Count by age')

In [ ]:
mouse_line_colors = ['#8da0cb', '#fc8d62', '#66c2a5']

In [ ]:
structure = 'root'
structure_id = ia_map[structure]
order = ['4 mo', '6 mo', '9 mo', '12 mo', '15 mo', '18 mo']

ages = []
groups = []
plaque_density = []
isids_carrythrough = []
for age in dataset['age group'].unique():
    for group in dataset[dataset['age group'] == age]['Mouse Line'].unique():
        isids = dataset[(dataset['Mouse Line'] == group) & (dataset['age group'] == age)]['image_series_id'].values
        for isid in isids:
            ages.append(age)
            groups.append(group)
            plaque_density.append(dat[(dat['structure_id'] == structure_id) & 
                                         (dat['image_series_id'] == isid)]
                                     ['plaque_density'].values[0])
            isids_carrythrough.append(isid)
density_dat = pd.DataFrame({'age': ages, 'structure': structure, 
                             'group': groups, 'plaque density': plaque_density,
                             'image_series_id': isids_carrythrough})
density_dat['plaque density'] = [density*100 for density in density_dat['plaque density']]

ages = []
groups = []
plaque_count = []
isids_carrythrough = []
for age in dataset['age group'].unique():
    for group in dataset[dataset['age group'] == age]['Mouse Line'].unique():
        isids = dataset[(dataset['Mouse Line'] == group) & (dataset['age group'] == age)]['image_series_id'].values
        for isid in isids:
            ages.append(age)
            groups.append(group)
            plaque_count.append(dat_count[(dat_count['structure_id'] == structure_id) & 
                                         (dat_count['image_series_id'] == isid)]
                                     ['count_per_mm_3'].values[0])
            isids_carrythrough.append(isid)
count_dat = pd.DataFrame({'age': ages, 'structure': structure, 
                             'group': groups, 'plaque count': plaque_count,
                             'image_series_id': isids_carrythrough})

fig, ax = plt.subplots(1, 2, figsize = (18, 7))
g = sns.swarmplot('age', 'plaque density', hue = 'group', data=density_dat, ax=ax[0], s=7, alpha = 0.8,
                 order = order, palette = mouse_line_colors)
#g.set_ylim([0, 0.016])
g.legend_.remove()
g.set_ylabel('Plaque Percent Volume')
g.set_title('Brain-Wide Plaque Density')
h = sns.swarmplot('age', 'plaque count', hue = 'group', 
                  data=count_dat, ax=ax[1], s=7, alpha = 0.8, order = order,
                 palette = mouse_line_colors)
h.set_ylabel('Plaques Per mm$\mathregular{^{3}}$')
h.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
h.set_title('Brain-Wide Plaque Count')
fig.subplots_adjust(wspace = 0.3)

In [ ]:
structure = 'root'
structure_id = ia_map[structure]
order = ['4 mo', '6 mo', '9 mo', '12 mo', '15 mo', '18 mo']

ages = []
groups = []
plaque_density = []
isids_carrythrough = []
for age in dataset['age group'].unique():
    for group in dataset[dataset['age group'] == age]['Mouse Line'].unique():
        if group != 'APP/PS1/rTg4510':
            isids = dataset[(dataset['Mouse Line'] == group) & (dataset['age group'] == age)]['image_series_id'].values
            for isid in isids:
                ages.append(age)
                groups.append(group)
                plaque_density.append(dat[(dat['structure_id'] == structure_id) & 
                                             (dat['image_series_id'] == isid)]
                                         ['plaque_density'].values[0])
                isids_carrythrough.append(isid)
density_dat = pd.DataFrame({'age': ages, 'structure': structure, 
                             'group': groups, 'plaque density': plaque_density,
                             'image_series_id': isids_carrythrough})
density_dat['plaque density'] = [density*100 for density in density_dat['plaque density']]

ages = []
groups = []
plaque_count = []
isids_carrythrough = []
for age in dataset['age group'].unique():
    for group in dataset[dataset['age group'] == age]['Mouse Line'].unique():
        if group != 'APP/PS1/rTg4510':
            isids = dataset[(dataset['Mouse Line'] == group) & (dataset['age group'] == age)]['image_series_id'].values
            for isid in isids:
                ages.append(age)
                groups.append(group)
                plaque_count.append(dat_count[(dat_count['structure_id'] == structure_id) & 
                                             (dat_count['image_series_id'] == isid)]
                                         ['count_per_mm_3'].values[0])
                isids_carrythrough.append(isid)
count_dat = pd.DataFrame({'age': ages, 'structure': structure, 
                             'group': groups, 'plaque count': plaque_count,
                             'image_series_id': isids_carrythrough})

fig, ax = plt.subplots(1, 2, figsize = (18, 7))
g = sns.swarmplot('age', 'plaque density', hue = 'group', data=density_dat, ax=ax[0], s=10,
                 order = order, palette = mouse_line_colors, hue_order = ['APP/PS1', 'hAPP-J20', 'Tg2576'])
#g.set_ylim([0, 0.016])
g.legend_.remove()
g.set_ylabel('Plaque Percent Volume')
g.set_title('Brain-Wide Plaque Density')
h = sns.swarmplot('age', 'plaque count', hue = 'group', 
                  data=count_dat, ax=ax[1], s=10, order = order,
                 palette = mouse_line_colors, hue_order = ['APP/PS1', 'hAPP-J20', 'Tg2576'])
h.set_ylabel('Plaques Per mm$\mathregular{^{3}}$')
g.legend(loc=2, borderaxespad=0.)
h.legend(loc=2, borderaxespad=0.)
h.set_title('Brain-Wide Plaque Count')
fig.subplots_adjust(wspace = 0.3)
plt.tight_layout()
plt.savefig(os.path.join(savepath, 'plaque density count brain-wide 3 lines.pdf'), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=1000)

In [ ]:
np.sqrt(600)

In [ ]:
def find_density_by_structure(structure):
    structure_id = ia_map[structure]
    ages = []
    groups = []
    plaque_density = []
    isids_carrythrough = []
    for age in dataset['age group'].unique():
        for group in dataset[dataset['age group'] == age]['Mouse Line'].unique():
            isids = dataset[(dataset['Mouse Line'] == group) & (dataset['age group'] == age)]['image_series_id'].values
            for isid in isids:
                ages.append(age)
                groups.append(group)
                plaque_density.append(dat[(dat['structure_id'] == structure_id) & 
                                             (dat['image_series_id'] == isid)]
                                         ['plaque_density'].values[0])
                isids_carrythrough.append(isid)
    density_dat = pd.DataFrame({'age': ages, 'structure': structure, 
                                 'group': groups, 'plaque density': plaque_density,
                                 'image_series_id': isids_carrythrough})
    density_dat['plaque density'] = [density*100 for density in density_dat['plaque density']]
    return(density_dat)

def find_count_by_structure(structure):
    structure_id = ia_map[structure]
    ages = []
    groups = []
    plaque_count = []
    isids_carrythrough = []
    for age in dataset['age group'].unique():
        for group in dataset[dataset['age group'] == age]['Mouse Line'].unique():
            isids = dataset[(dataset['Mouse Line'] == group) & (dataset['age group'] == age)]['image_series_id'].values
            for isid in isids:
                ages.append(age)
                groups.append(group)
                plaque_count.append(dat_count[(dat_count['structure_id'] == structure_id) & 
                                             (dat_count['image_series_id'] == isid)]
                                         ['count_per_mm_3'].values[0])
                isids_carrythrough.append(isid)
    count_dat = pd.DataFrame({'age': ages, 'structure': structure, 
                                 'group': groups, 'plaque count': plaque_count,
                                 'image_series_id': isids_carrythrough})
    return(count_dat)

In [ ]:
order = ['4 mo', '6 mo', '9 mo', '12 mo', '15 mo', '18 mo']
root_d = find_density_by_structure('root')
root_c = find_count_by_structure('root')

iso_d = find_density_by_structure('Isocortex')
iso_c = find_count_by_structure('Isocortex')

hpc_d = find_density_by_structure('HPF')
hpc_c = find_count_by_structure('HPF')

fig, ax = plt.subplots(3, 2, sharex = True, figsize = (18, 20))
a = sns.swarmplot('age', 'plaque density', hue = 'group', data=root_d, ax=ax[0, 0], s=7,
                 order = order, palette = mouse_line_colors)
a.set_ylabel('Plaque % Volume')
a.legend(loc=2, borderaxespad=0.)
a.set_title('Brain-Wide Plaque Density')
b = sns.swarmplot('age', 'plaque count', hue = 'group', data=root_c, ax=ax[0, 1], s=7,
                  order = order, palette = mouse_line_colors)
b.set_ylabel('Plaques Per mm$\mathregular{^{3}}$')
b.set_title('Brain-Wide Plaque Count')
b.legend_.remove()

c = sns.swarmplot('age', 'plaque density', hue = 'group', data=iso_d, ax=ax[1, 0], s=7,
                 order = order, palette = mouse_line_colors)
c.set_ylabel('Plaque % Volume')
c.legend_.remove()
c.set_title('Plaque Density in Isocortex')
d = sns.swarmplot('age', 'plaque count', hue = 'group', data=iso_c, ax=ax[1, 1], s=7,
                  order = order, palette = mouse_line_colors)
d.set_ylabel('Plaques Per mm$\mathregular{^{3}}$')
d.legend_.remove()
d.set_title('Plaque Count in Isocortex')

e = sns.swarmplot('age', 'plaque density', hue = 'group', data=hpc_d, ax=ax[2, 0], s=7,
                 order = order, palette = mouse_line_colors)
e.set_ylabel('Plaque % Volume')
e.legend_.remove()
e.set_title('Plaque Density in Hippocampus')
f = sns.swarmplot('age', 'plaque count', hue = 'group', data=hpc_c, ax=ax[2, 1], s=7,
                  order = order, palette = mouse_line_colors)
f.set_ylabel('Plaques Per mm$\mathregular{^{3}}$')
f.legend_.remove()
f.set_title('Plaque Count in Hippocampus')

plt.tight_layout()
plt.savefig(os.path.join(savepath, 'plaque density count brain-wide iso hipp 3 lines.pdf'), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=1000)

In [ ]:
density_dat[density_dat['age'] == '15 mo'][['image_series_id', 'plaque density']].sort_values(by = 'plaque density')

In [ ]:
order = ['4 mo', '6 mo', '9 mo', '12 mo', '15 mo', '18 mo']
root_d = find_density_by_structure('root')
fig, ax = plt.subplots(1, figsize = (15, 5))
a = sns.swarmplot('age', 'plaque density', hue = 'group', 
                  data=root_d[root_d['group'].isin(['APP/PS1', 'hAPP-J20', 'Tg2576'])], 
                  s=7, alpha = 0.8, ax = ax, order = order, palette = mouse_line_colors)
a.set_ylabel('Plaque % Volume')
a.legend(loc=2, borderaxespad=0.)
plt.savefig(os.path.join(savepath, 'whole brain plaque density by age 3 lines scatter.pdf'), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=1000)

In [ ]:
order = ['4 mo', '6 mo', '9 mo', '12 mo', '15 mo', '18 mo']
root_d = find_count_by_structure('root')
fig, ax = plt.subplots(1, figsize = (15, 5))
a = sns.swarmplot('age', 'plaque count', hue = 'group', 
                  data=root_c[root_c['group'].isin(['APP/PS1', 'hAPP-J20', 'Tg2576'])], 
                  s=7, alpha = 0.8, ax = ax, order = order, palette = mouse_line_colors)
a.set_ylabel('Plaques Per mm$\mathregular{^{3}}$')
a.legend(loc=2, borderaxespad=0.)
plt.savefig(os.path.join(savepath, 'whole brain plaque count by age 3 lines scatter.pdf'), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=1000)

In [ ]:
g = sns.factorplot('age', 'plaque count', hue = 'group', 
                   data=count_dat, s=7, alpha = 0.8, size = 5, aspect = 2,
                  order = ['4 mo', '6 mo', '9 mo', '12 mo', '15 mo', '18 mo'])
g.set(yscale="log")
plt.title('Whole Brain Plaque Count')

In [ ]:
density_dat.head()

In [ ]:
density_dat = find_density_by_structure('root')
g = sns.factorplot('age', 'plaque density', hue = 'group', 
                   data=density_dat[density_dat['group'] != 'APP/PS1/rTg4510'], s=7, 
                   size = 5, aspect = 4, order = ['4 mo', '6 mo', '9 mo', '12 mo', '15 mo', '18 mo'],
                  palette = mouse_line_colors, legend = False)
g.set(yscale="log")
ax.set_ylim([0.001, 10])
g.set_yticklabels(['0.001', '0.01', '0.1', '1', '10'])
plt.title('Whole Brain Plaque Density')
plt.legend(loc='upper left')
ax = g.ax
ax.set_ylabel('Percent Plaque Coverage')
ax.set_xlabel('')
plt.savefig(os.path.join(savepath, 'whole brain plaque density by age 3 lines.pdf'), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=1000)

In [ ]:
len(density_dat[(density_dat['age'] == '15 mo') & (density_dat['group'] == 'hAPP-J20')])

In [ ]:
density_dat = find_density_by_structure('HPF')
g = sns.factorplot('age', 'plaque density', hue = 'group', 
                   data=density_dat[density_dat['group'] != 'APP/PS1/rTg4510'], s=7, alpha = 0.8, 
                   size = 5, aspect = 3, order = ['4 mo', '6 mo', '9 mo', '12 mo', '15 mo', '18 mo'],
                  palette = mouse_line_colors)
g.set(yscale="log")
ax.set_ylim([0.001, 10])
g.set_yticklabels(['0.001', '0.01', '0.1', '1', '10'])
plt.title('Hippocampus Plaque Density')
plt.savefig(os.path.join(savepath, 'hippocampus plaque density by age 3 lines.pdf'), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=1000)

In [ ]:
density_dat = find_density_by_structure('Isocortex')
g = sns.factorplot('age', 'plaque density', hue = 'group', 
                   data=density_dat[density_dat['group'] != 'APP/PS1/rTg4510'], s=7, alpha = 0.8, 
                   size = 5, aspect = 3, order = ['4 mo', '6 mo', '9 mo', '12 mo', '15 mo', '18 mo'],
                  palette = mouse_line_colors)
g.set(yscale="log")
ax.set_ylim([0.001, 10])
g.set_yticklabels(['0.001', '0.01', '0.1', '1', '10'])
plt.title('Isocortex Plaque Density')
plt.savefig(os.path.join(savepath, 'isocortex plaque density by age 3 lines.pdf'), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=1000)

In [ ]:
dat_count.head()

In [ ]:
structure = 'root'
structure_id = ia_map[structure]
ages = []
groups = []
plaque_count = []
isids_carrythrough = []
for age in dataset['age group'].unique():
    for group in dataset[dataset['age group'] == age]['Mouse Line'].unique():
        isids = dataset[(dataset['Mouse Line'] == group) & (dataset['age group'] == age)]['image_series_id'].values
        for isid in isids:
            ages.append(age)
            groups.append(group)
            plaque_count.append(dat_count[(dat_count['structure_id'] == structure_id) & 
                                         (dat_count['image_series_id'] == isid)]
                                     ['count_per_mm_3'].values[0])
            isids_carrythrough.append(isid)
count_dat = pd.DataFrame({'age': ages, 'structure': structure, 
                             'group': groups, 'plaque count': plaque_count,
                             'image_series_id': isids_carrythrough})
g = sns.factorplot('age', 'plaque count', hue = 'group', 
                   data=count_dat[count_dat['group'] != 'APP/PS1/rTg4510'], s=7, alpha = 0.8, 
                   size = 5, aspect = 4, order = ['4 mo', '6 mo', '9 mo', '12 mo', '15 mo', '18 mo'],
                  palette = mouse_line_colors, legend = False)
g.set_ylabels('Plaques Per mm$\mathregular{^{3}}$')
g.set(yscale="log")
plt.legend(loc='upper left')
plt.title('Whole Brain Plaque Load')
plt.savefig(os.path.join(savepath, 'whole brain plaque count by age 3 lines.pdf'), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=1000)

In [ ]:
structure = 'Isocortex'
structure_id = ia_map[structure]
ages = []
groups = []
plaque_count = []
isids_carrythrough = []
for age in dataset['age group'].unique():
    for group in dataset[dataset['age group'] == age]['Mouse Line'].unique():
        isids = dataset[(dataset['Mouse Line'] == group) & (dataset['age group'] == age)]['image_series_id'].values
        for isid in isids:
            ages.append(age)
            groups.append(group)
            plaque_count.append(dat_count[(dat_count['structure_id'] == structure_id) & 
                                         (dat_count['image_series_id'] == isid)]
                                     ['count_per_mm_3'].values[0])
            isids_carrythrough.append(isid)
count_dat = pd.DataFrame({'age': ages, 'structure': structure, 
                             'group': groups, 'plaque count': plaque_count,
                             'image_series_id': isids_carrythrough})
g = sns.factorplot('age', 'plaque count', hue = 'group', 
                   data=count_dat[count_dat['group'] != 'APP/PS1/rTg4510'], s=7, alpha = 0.8, 
                   size = 5, aspect = 3, order = ['4 mo', '6 mo', '9 mo', '12 mo', '15 mo', '18 mo'],
                  palette = mouse_line_colors)
g.set(yscale="log")
plt.title('Isocortex Plaque Count')

In [ ]:
h='70FF71'
tuple(int(h[i:i+2], 16) for i in (0, 2 ,4))

In [ ]:
'#{:02x}{:02x}{:02x}'.format( 204, 204, 204)

In [ ]:
ccf_colors = ['#70ff71', '#9ad2bd', '#7ed04b', '#8ada87', '#98d6f9', '#8599cc', '#ff7080', '#e64438', '#ff64ff', 
              '#ff9b88','#ff9bcd', '#f0f080', '#cccccc']

In [ ]:
oo_dict_coarse

In [ ]:
pltdat = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
for structure in coarse_structure_ids:
    structure_acronym = ai_map[structure]
    pltdat = pd.concat([pltdat, get_structure_data('9 mo', structure_acronym, 'density')])
mdf = pd.melt(pltdat, id_vars = ['age', 'structure', 'group', 'image_series_id'], value_name = 'plaque density')
mdf['plaque density'] = [density*100 for density in mdf['plaque density']]
mdf['ont_order'] = [oo_dict_coarse[acronym] for acronym in mdf['structure']]
mdf = mdf.sort_values(by='ont_order')
fig, ax = plt.subplots(1, 3, figsize = (10, 10), sharex = False, sharey = True)
groups = ['APP/PS1', 'hAPP-J20', 'Tg2576'] # same as mdf['group'].unique() but keeps consistent plot order
for ix in range(3):
    group = groups[ix]
    g = sns.boxplot('plaque density', 'structure', data = mdf[mdf['group'] == group], ax=ax[ix], 
                    color = 'white')
    sns.swarmplot('plaque density', 'structure', data = mdf[mdf['group'] == group], s=8, alpha = 0.8,
                  palette = ccf_colors, ax=ax[ix])
    g.set_title(group)
    if group in ['hAPP-J20', 'Tg2576']:
        g.set_xlim([0, 0.7])
        ax[ix].xaxis.set_major_locator(ticker.MultipleLocator(0.6))
        ax[ix].xaxis.set_major_formatter(ticker.ScalarFormatter())
        ax[ix].set_ylabel('')
        ax[ix].set_xlabel('')
    if group == 'APP/PS1':
        g.set_xlim([0, 3])
        ax[ix].xaxis.set_major_locator(ticker.MultipleLocator(2.5))
        ax[ix].xaxis.set_major_formatter(ticker.ScalarFormatter())
        ax[ix].set_ylabel('')
        ax[ix].set_xlabel('')
    if group == 'hAPP-J20':
        ax[ix].set_xlabel('Percent Plaque Coverage')
    plt.tight_layout()
    
fig.subplots_adjust(wspace = 0.3)
plt.savefig(os.path.join(savepath, 'box-swarm plots major divisions 9 mo.pdf'), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=1000)

In [ ]:
mdf[(mdf['age'] == '9 mo') & (mdf['group'] == 'hAPP-J20') & (mdf['structure'] == 'MY')].sort_values(by = 'plaque density')

In [ ]:
pltdat = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
for structure in coarse_structure_ids:
    structure_acronym = ai_map[structure]
    pltdat = pd.concat([pltdat, get_structure_data('12 mo', structure_acronym, 'density')])
mdf = pd.melt(pltdat, id_vars = ['age', 'structure', 'group', 'image_series_id'], value_name = 'plaque density')
mdf['plaque density'] = [density*100 for density in mdf['plaque density']]
mdf['ont_order'] = [oo_dict_coarse[acronym] for acronym in mdf['structure']]
mdf = mdf.sort_values(by='ont_order')
fig, ax = plt.subplots(1, 3, figsize = (10, 10), sharex = False, sharey = True)
groups = ['APP/PS1', 'hAPP-J20', 'Tg2576'] # same as mdf['group'].unique() but keeps consistent plot order
for ix in range(3):
    group = groups[ix]
    g = sns.boxplot('plaque density', 'structure', data = mdf[mdf['group'] == group], ax=ax[ix], 
                    color = 'white')
    sns.swarmplot('plaque density', 'structure', data = mdf[mdf['group'] == group], s=8, alpha = 0.8,
                  palette = ccf_colors, ax=ax[ix])
    g.set_title(group)
    if group in ['hAPP-J20', 'Tg2576']:
        g.set_xlim([0, 0.7])
        ax[ix].xaxis.set_major_locator(ticker.MultipleLocator(0.6))
        ax[ix].xaxis.set_major_formatter(ticker.ScalarFormatter())
        ax[ix].set_ylabel('')
        ax[ix].set_xlabel('')
    if group == 'APP/PS1':
        g.set_xlim([0, 3])
        ax[ix].xaxis.set_major_locator(ticker.MultipleLocator(2.5))
        ax[ix].xaxis.set_major_formatter(ticker.ScalarFormatter())
        ax[ix].set_ylabel('')
        ax[ix].set_xlabel('')
    if group == 'hAPP-J20':
        ax[ix].set_xlabel('Percent Plaque Coverage')
    plt.tight_layout()
    fig.subplots_adjust(wspace = 0.3)
plt.savefig(os.path.join(savepath, 'box-swarm plots major divisions 12 mo.pdf'), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=1000)

In [ ]:
mdf[(mdf['age'] == '12 mo') & (mdf['group'] == 'hAPP-J20') & (mdf['structure'] == 'fiber tracts')].sort_values(by = 'plaque density')

In [ ]:
pltdat = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
for structure in coarse_structure_ids:
    structure_acronym = ai_map[structure]
    pltdat = pd.concat([pltdat, get_structure_data('18 mo', structure_acronym, 'density')])
mdf = pd.melt(pltdat, id_vars = ['age', 'structure', 'group', 'image_series_id'], value_name = 'plaque density')
mdf['plaque density'] = [density*100 for density in mdf['plaque density']]
mdf['ont_order'] = [oo_dict_coarse[acronym] for acronym in mdf['structure']]
mdf = mdf.sort_values(by='ont_order')
fig, ax = plt.subplots(1, 3, figsize = (10, 10), sharex = False, sharey = True)
groups = ['APP/PS1', 'hAPP-J20', 'Tg2576'] # same as mdf['group'].unique() but keeps consistent plot order
for ix in range(3):
    group = groups[ix]
    g = sns.boxplot('plaque density', 'structure', data = mdf[mdf['group'] == group], ax=ax[ix], 
                    color = 'white')
    sns.swarmplot('plaque density', 'structure', data = mdf[mdf['group'] == group], s=8, alpha = 0.8,
                  palette = ccf_colors, ax=ax[ix])
    g.set_title(group)
    if group in ['hAPP-J20', 'Tg2576']:
        g.set_xlim([0, 0.7])
        ax[ix].xaxis.set_major_locator(ticker.MultipleLocator(0.6))
        ax[ix].xaxis.set_major_formatter(ticker.ScalarFormatter())
        ax[ix].set_ylabel('')
        ax[ix].set_xlabel('')
    if group == 'APP/PS1':
        g.set_xlim([0, 3])
        ax[ix].xaxis.set_major_locator(ticker.MultipleLocator(2.5))
        ax[ix].xaxis.set_major_formatter(ticker.ScalarFormatter())
        ax[ix].set_ylabel('')
        ax[ix].set_xlabel('')
    if group == 'hAPP-J20':
        ax[ix].set_xlabel('Percent Plaque Coverage')
    plt.tight_layout()
    fig.subplots_adjust(wspace = 0.3)
plt.savefig(os.path.join(savepath, 'box-swarm plots major divisions 18 mo.pdf'), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=1000)

In [ ]:
mdf[(mdf['age'] == '18 mo') & (mdf['group'] == 'Tg2576') & (mdf['structure'] == 'OLF')].sort_values(by = 'plaque density')

In [ ]:
pltdat_9 = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
for structure in coarse_structure_ids:
    structure_acronym = ai_map[structure]
    pltdat_9 = pd.concat([pltdat_9, get_structure_data('9 mo', structure_acronym, 'density')])
mdf_9 = pd.melt(pltdat_9, id_vars = ['age', 'structure', 'group', 'image_series_id'], value_name = 'plaque density')
mdf_9['ont_order'] = [oo_dict_coarse[acronym] for acronym in mdf_9['structure']]
mdf_9 = mdf_9.sort_values(by='ont_order')

pltdat_12 = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
for structure in coarse_structure_ids:
    structure_acronym = ai_map[structure]
    pltdat_12 = pd.concat([pltdat_12, get_structure_data('12 mo', structure_acronym, 'density')])
mdf_12 = pd.melt(pltdat_12, id_vars = ['age', 'structure', 'group', 'image_series_id'], value_name = 'plaque density')
mdf_12['ont_order'] = [oo_dict_coarse[acronym] for acronym in mdf_12['structure']]
mdf_12 = mdf_12.sort_values(by='ont_order')

pltdat_18 = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
for structure in coarse_structure_ids:
    structure_acronym = ai_map[structure]
    pltdat_18 = pd.concat([pltdat_18, get_structure_data('18 mo', structure_acronym, 'density')])
mdf_18 = pd.melt(pltdat_18, id_vars = ['age', 'structure', 'group', 'image_series_id'], 
                 value_name = 'plaque density')
mdf_18['ont_order'] = [oo_dict_coarse[acronym] for acronym in mdf_18['structure']]
mdf_18 = mdf_18.sort_values(by='ont_order')

In [ ]:
mdf_9[mdf_9['plaque density'].isnull()]

In [ ]:
mouse_line_colors

In [ ]:
mdf_12.group.unique()

In [ ]:
pltdat = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
for structure in coarse_structure_ids:
    structure_acronym = ai_map[structure]
    pltdat = pd.concat([pltdat, get_structure_data('18 mo', structure_acronym, 'count')])
mdf = pd.melt(pltdat, id_vars = ['age', 'structure', 'group', 'image_series_id'], value_name = 'plaque count')
mdf['ont_order'] = [oo_dict_coarse[acronym] for acronym in mdf['structure']]
mdf = mdf.sort_values(by='ont_order')
fig, ax = plt.subplots(1, 3, figsize = (10, 10), sharex = False, sharey = True)
groups = ['APP/PS1', 'hAPP-J20', 'Tg2576'] # same as mdf['group'].unique() but keeps consistent plot order
for ix in range(3):
    group = groups[ix]
    g = sns.boxplot('plaque count', 'structure', data = mdf[mdf['group'] == group], ax=ax[ix], 
                    color = 'white')
    sns.swarmplot('plaque count', 'structure', data = mdf[mdf['group'] == group], s=8, alpha = 0.8,
                  palette = ccf_colors, ax=ax[ix])
    g.set_title(group)
    if group in ['hAPP-J20', 'Tg2576']:
        ax[ix].set_xlabel('')
        ax[ix].set_ylabel('')
        g.set_xlim([0, 500])
        ax[ix].xaxis.set_major_locator(ticker.MultipleLocator(500))
        ax[ix].xaxis.set_major_formatter(ticker.ScalarFormatter())
    if group == 'APP/PS1':
        ax[ix].set_xlabel('')
        ax[ix].set_ylabel('')
        g.set_xlim([0, 2500])
        ax[ix].xaxis.set_major_locator(ticker.MultipleLocator(2500))
        ax[ix].xaxis.set_major_formatter(ticker.ScalarFormatter())
    if group == 'hAPP-J20':
        ax[ix].set_xlabel('Plaques per mm$\mathregular{^{3}}$')
    plt.tight_layout()
plt.savefig(os.path.join(savepath, 'box-swarm plots plaque count major divisions 18 mo.pdf'), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=1000)

In [ ]:
pltdat = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
for structure in coarse_structure_ids:
    structure_acronym = ai_map[structure]
    pltdat = pd.concat([pltdat, get_structure_data('12 mo', structure_acronym, 'count')])
mdf = pd.melt(pltdat, id_vars = ['age', 'structure', 'group', 'image_series_id'], value_name = 'plaque count')
mdf['ont_order'] = [oo_dict_coarse[acronym] for acronym in mdf['structure']]
mdf = mdf.sort_values(by='ont_order')
fig, ax = plt.subplots(1, 3, figsize = (10, 10), sharex = False, sharey = True)
groups = ['APP/PS1', 'hAPP-J20', 'Tg2576'] # same as mdf['group'].unique() but keeps consistent plot order
for ix in range(3):
    group = groups[ix]
    g = sns.boxplot('plaque count', 'structure', data = mdf[mdf['group'] == group], ax=ax[ix], 
                    color = 'white')
    sns.swarmplot('plaque count', 'structure', data = mdf[mdf['group'] == group], s=8, alpha = 0.8,
                  palette = ccf_colors, ax=ax[ix])
    g.set_title(group)
    if group in ['hAPP-J20', 'Tg2576']:
        ax[ix].set_xlabel('')
        ax[ix].set_ylabel('')
        g.set_xlim([0, 500])
        ax[ix].xaxis.set_major_locator(ticker.MultipleLocator(500))
        ax[ix].xaxis.set_major_formatter(ticker.ScalarFormatter())
    if group == 'APP/PS1':
        ax[ix].set_xlabel('')
        ax[ix].set_ylabel('')
        g.set_xlim([0, 2500])
        ax[ix].xaxis.set_major_locator(ticker.MultipleLocator(2500))
        ax[ix].xaxis.set_major_formatter(ticker.ScalarFormatter())
    if group == 'hAPP-J20':
        ax[ix].set_xlabel('Plaques per mm$\mathregular{^{3}}$')
    plt.tight_layout()
plt.savefig(os.path.join(savepath, 'box-swarm plots plaque count major divisions 12 mo.pdf'), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=1000)

In [ ]:
order = ['APP/PS1', 'hAPP-J20', 'Tg2576']
fig, ax = plt.subplots(1, 3, figsize = (10, 10), sharey = True)
g = sns.swarmplot('plaque density', 'structure', data = mdf_9, hue = 'group', s=8, 
                  ax = ax[0], palette = mouse_line_colors)
g.set_title('9 mo')
ax[0].set_xlim([0, 0.035])
ax[0].legend_.remove()
ax[0].xaxis.set_major_locator(ticker.MultipleLocator(0.05))
ax[0].xaxis.set_major_formatter(ticker.ScalarFormatter())

h = sns.swarmplot('plaque density', 'structure', data = mdf_12, hue = 'group', s=8, 
                  ax = ax[1], palette = mouse_line_colors)
h.set_title('12 mo')
ax[1].set_xlim([0, 0.035])
ax[1].legend_.remove()
ax[1].xaxis.set_major_locator(ticker.MultipleLocator(0.05))
ax[1].xaxis.set_major_formatter(ticker.ScalarFormatter())

i = sns.swarmplot('plaque density', 'structure', data = mdf_18, hue = 'group', s=8, 
                  ax = ax[2], palette = mouse_line_colors)
i.set_title('18 mo')
ax[2].set_xlim([0, 0.035])
ax[2].legend_.remove()
ax[2].xaxis.set_major_locator(ticker.MultipleLocator(0.05))
ax[2].xaxis.set_major_formatter(ticker.ScalarFormatter())

plt.tight_layout()

In [ ]:
mdf.head()

In [ ]:
mdf[(mdf['group'] == 'hAPP-J20') & (mdf['age'] == '9 mo') & (mdf['structure'] == 'Isocortex')].sort_values(by = 'plaque count')

In [ ]:
olf = structure_tree.get_structures_by_acronym(['OLF'])[0]
olf = structure_tree.descendant_ids([olf['id']])[0]
olf = [structure for structure in olf if structure in summary_structure_ids]

In [ ]:
iso = structure_tree.get_structures_by_acronym(['Isocortex'])[0]
iso = structure_tree.descendant_ids([iso['id']])[0]
iso = [structure for structure in iso if structure in summary_structure_ids]

In [ ]:
plot_order = ['FRP', 'MOp', 'MOs', 'SSp-n', 'SSp-bfd', 'SSp-ll', 'SSp-m', 'SSp-ul', 'SSp-tr', 'SSp-un',
              'SSs', 'GU', 'VISC', 'AUDd', 'AUDp', 'AUDpo', 'AUDv', 'VISal', 'VISam', 'VISl', 'VISp', 'VISpl',
              'VISpm', 'VISli', 'VISpor', 'ACAd', 'ACAv', 'PL', 'ILA', 'ORBl', 'ORBm', 'ORBvl', 'AId', 'AIp',
              'AIv', 'RSPagl', 'RSPd', 'RSPv', 'VISa', 'VISrl', 'TEa', 'PERI', 'ECT']

In [ ]:
pltdat.head()

In [ ]:
pltdat = pd.DataFrame(columns={'age', 'group', 'plaque density', 'structure'})
for structure in iso:
    structure_acronym = ai_map[structure]
    pltdat = pd.concat([pltdat, get_structure_data('12 mo', structure_acronym, 'count')])
mdf = pd.melt(pltdat, id_vars = ['age', 'structure', 'group', 'image_series_id'], value_name = 'plaque count')
g = sns.factorplot('structure', 'plaque count', hue = 'group', 
                   data = mdf[mdf['group'].isin(['APP/PS1', 'APP/PS1/rTg4510'])],
                   order = plot_order, size=8, aspect=16 / 8.)
plt.xticks(rotation = 90)
plt.tight_layout()

In [ ]:
pltdat = pd.DataFrame(columns={'age', 'group', 'plaque density', 'structure'})
for structure in iso:
    structure_acronym = ai_map[structure]
    pltdat = pd.concat([pltdat, get_structure_data('12 mo', structure_acronym, 'count')])
mdf = pd.melt(pltdat, id_vars = ['age', 'structure', 'group', 'image_series_id'], value_name = 'plaque count')
g = sns.factorplot('structure', 'plaque count', hue = 'group', 
                   data = mdf[mdf['group'].isin(['APP/PS1', 'Tg2576', 'hAPP-J20'])], size=8, aspect=16 / 8, 
                   order = plot_order, legend=False, palette = mouse_line_colors)
g.set(yscale = 'log')
plt.xticks(rotation = 90)
plt.legend(loc=3)
plt.tight_layout()

In [ ]:
len(mdf[(mdf['group'] == 'APP/PS1') & (mdf['structure'] == 'ACAd')]['image_series_id'].unique())

In [ ]:
len(ss_acronyms)

In [ ]:
'ProS' in ss_acronyms

In [ ]:
iso_strs = [ai_map[structure] for structure in iso]

In [ ]:
ctxdat = dat[(dat['image_series_id'].isin(dataset['Link to image series'])) &
             (dat['structure_acronym'].isin(iso_strs))]

In [ ]:
ctxdat.sort_values(by = 'structure_acronym', inplace = True)

In [ ]:
ctxdat.to_csv(os.path.join(path, 'plaque_density_ctx_structures.csv'))

In [ ]:
pltdat = pd.DataFrame(columns={'age', 'group', 'plaque density', 'structure'})
for structure in iso_strs:
    pltdat = pd.concat([pltdat, get_structure_data('12 mo', structure, 'density')])
mdf = pd.melt(pltdat, id_vars = ['age', 'structure', 'group', 'image_series_id'], value_name = 'plaque density')
mdf['plaque density'] = [density*100 for density in mdf['plaque density']]
g = sns.factorplot('structure', 'plaque density', hue = 'group', 
                   data = mdf[mdf['group'].isin(['APP/PS1', 'APP/PS1/rTg4510'])],
                   order = plot_order, size=8, aspect=16 / 8.)
plt.xticks(rotation = 90)
plt.tight_layout()

In [ ]:
def get_structure_data(age, structure, q_type):
    if q_type == 'density':
        data = dat
        data = data.rename(columns = {'plaque_density': 'plaque'})
    elif q_type == 'count':
        data = dat_count
        data = data.rename(columns = {'count_per_mm_3': 'plaque'})    
    structure_id = ia_map[structure]
    groups = []
    plaques = []
    isids_carrythrough = []
    for group in dataset[dataset['age group'] == age]['Mouse Line'].unique():
        isids = dataset[(dataset['Mouse Line'] == group) & (dataset['age group'] == age)]['image_series_id'].values
        for isid in isids:
            groups.append(group)
            if len(data[(data['structure_id'] == structure_id) & (
                data['image_series_id'] == isid)]['plaque']) > 0:
                plaques.append(data[(data['structure_id'] == structure_id) & 
                                         (data['image_series_id'] == isid)]
                                     ['plaque'].values[0])
            else:
                print(isid)
                print(ai_map[structure_id])
                plaques.append(0)
                    
            isids_carrythrough.append(isid)
    structuredat = pd.DataFrame({'age': age, 'structure': structure, 
                                 'group': groups, 'plaques': plaques,
                                'image_series_id': isids_carrythrough})
    return structuredat

In [ ]:
pltdat = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
for structure in iso:
    structure_acronym = ai_map[structure]
    pltdat = pd.concat([pltdat, get_structure_data('18 mo', structure_acronym, 'density')])
mdf = pd.melt(pltdat, id_vars = ['age', 'structure', 'group', 'image_series_id'], value_name = 'plaque density')
mdf['plaque density'] = [density*100 for density in mdf['plaque density']]

fig, ax = plt.subplots(figsize = (16, 6))
g = sns.pointplot('structure', 'plaque density', hue = 'group', 
                   data = mdf[mdf['group'] == 'APP/PS1'], dodge=False, 
                   order = plot_order, palette = mouse_line_colors,
                  hue_order = ['APP/PS1', 'hAPP-J20', 'Tg2576'], ax = ax)
plt.xticks(rotation = 90)
plt.title('Plaque Volume in Isocortex Structures (18 months)')
ax2 = plt.twinx()
h = sns.pointplot('structure', 'plaque density', hue = 'group', 
                   data = mdf[mdf['group'].isin(['Tg2576', 'hAPP-J20'])], 
                   order = plot_order, palette = mouse_line_colors, dodge = False, 
                  hue_order = ['APP/PS1', 'hAPP-J20', 'Tg2576'], ax = ax2)
plt.legend(loc=1, bbox_to_anchor = (0.95, 1))
ax.legend_.remove()
#ax.set_ylim([0, 3])
ax.set_xlabel("")
ax.set_ylabel("APP/PS1")
ax2.legend_.remove()
ax2.set_ylabel("hAPP-J20, Tg2576")
#ax2.set_ylim([0, 1.5])
ax.legend(loc=2)
plt.tight_layout()

plt.savefig(os.path.join(savepath, 'plaque volume in isocortex structures 18 mo.pdf'.format(structure)), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=1000)

In [ ]:
pltdat = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
for structure in iso:
    structure_acronym = ai_map[structure]
    pltdat = pd.concat([pltdat, get_structure_data('12 mo', structure_acronym, 'density')])
mdf = pd.melt(pltdat, id_vars = ['age', 'structure', 'group', 'image_series_id'], value_name = 'plaque density')
mdf['plaque density'] = [density*100 for density in mdf['plaque density']]

fig, ax = plt.subplots(figsize = (16, 6))
g = sns.pointplot('structure', 'plaque density', hue = 'group', 
                   data = mdf[mdf['group'] == 'APP/PS1'], dodge=False, 
                   order = plot_order, palette = mouse_line_colors,
                  hue_order = ['APP/PS1', 'hAPP-J20', 'Tg2576'], ax = ax)
plt.xticks(rotation = 90)
plt.title('Percent Plaque Coverage in Isocortex (12 months)')
ax2 = plt.twinx()
h = sns.pointplot('structure', 'plaque density', hue = 'group', 
                   data = mdf[mdf['group'].isin(['Tg2576', 'hAPP-J20'])], 
                   order = plot_order, palette = mouse_line_colors, dodge = False, 
                  hue_order = ['APP/PS1', 'hAPP-J20', 'Tg2576'], ax = ax2)
ax.legend_.remove()
#ax.set_ylim([0, 2])
ax.set_xlabel("")
ax.set_ylabel("APP/PS1")
ax2.legend_.remove()
ax2.set_ylabel("hAPP-J20, Tg2576")
ax2.set_ylim([0, 0.25])
ax.legend(loc=2, bbox_to_anchor = (0, 1.05))
plt.tight_layout()

plt.savefig(os.path.join(savepath, 'plaque volume in isocortex structures 12 mo.pdf'.format(structure)), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=1000)

In [ ]:
tau_dataset.head(1)

In [ ]:
pltdattau.head()

In [ ]:
pltdat = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
for structure in iso:
    structure_acronym = ai_map[structure]
    pltdat = pd.concat([pltdat, get_structure_data('12 mo', structure_acronym, 'density')])
mdf = pd.melt(pltdat, id_vars = ['age', 'structure', 'group', 'image_series_id'], value_name = 'plaque density')
mdf['plaque density'] = [density*100 for density in mdf['plaque density']]

fig, ax = plt.subplots(figsize = (16, 6))
g = sns.pointplot('structure', 'plaque density', hue = 'group', 
                   data = mdf[mdf['group'].isin(['APP/PS1', 'APP/PS1/rTg4510'])], dodge=False, 
                   order = plot_order, palette = ['#8da0cb'], ax = ax)

pltdattau = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
for structure in iso:
    structure_acronym = ai_map[structure]
    pltdattau = pd.concat([pltdattau, get_structure_data_tau('12 mo', structure_acronym, 'density')])
pltdattau = pltdattau[~pltdattau['image_series_id'].isin([682203974, 689724497, 693151433,
                                                         701806685, 715059578])] #Poor registration
mdftau = pd.melt(pltdattau, id_vars = ['age', 'structure', 'group', 'image_series_id'], 
                 value_name = 'plaque density')
mdftau['plaque density'] = [density*100 for density in mdftau['plaque density']]
h = sns.pointplot('structure', 'plaque density', hue = 'group', 
                   data = mdftau[mdftau['group'] == 'APP+/Tau-'], dodge=False, 
                   order = plot_order, palette = ['#52607f'], ax = ax, alpha = 0.8)
j = sns.swarmplot('structure', 'plaque density', data = mdftau[mdftau['group'] == 'APP+/Tau+'], 
                  order = plot_order, hue = 'group', s=15, ax = ax, palette = ['#C267FF'], alpha = 0.8)
plt.xticks(rotation = 90)
plt.title('Percent Plaque Coverage in Isocortex (12 months)')
ax.legend_.remove()
#ax.set_ylim([0, 2])
ax.set_xlabel("")
ax.set_ylabel("APP/PS1")
ax.legend(loc=2, bbox_to_anchor = (0.03, 1.05))
plt.tight_layout()

plt.savefig(os.path.join(savepath, 'plaque volume in isocortex structures 12 mo APP_PS1 and Tg4510.pdf'.format(structure)), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=1000)

In [ ]:
pltdat = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
for structure in iso:
    structure_acronym = ai_map[structure]
    pltdat = pd.concat([pltdat, get_structure_data('6 mo', structure_acronym, 'density')])
mdf = pd.melt(pltdat, id_vars = ['age', 'structure', 'group', 'image_series_id'], value_name = 'plaque density')
mdf['plaque density'] = [density*100 for density in mdf['plaque density']]

fig, ax = plt.subplots(figsize = (16, 6))
g = sns.pointplot('structure', 'plaque density', hue = 'group', 
                   data = mdf[mdf['group'].isin(['APP/PS1', 'APP/PS1/rTg4510'])], dodge=False, 
                   order = plot_order, palette = ['#8da0cb'], ax = ax)

pltdattau = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
for structure in iso:
    structure_acronym = ai_map[structure]
    pltdattau = pd.concat([pltdattau, get_structure_data_tau('6 mo', structure_acronym, 'density')])
pltdattau = pltdattau[~pltdattau['image_series_id'].isin([682203974, 689724497, 693151433,
                                                         701806685, 715059578])] #Poor registration
mdftau = pd.melt(pltdattau, id_vars = ['age', 'structure', 'group', 'image_series_id'], 
                 value_name = 'plaque density')
mdftau['plaque density'] = [density*100 for density in mdftau['plaque density']]
h = sns.pointplot('structure', 'plaque density', hue = 'group', 
                   data = mdftau[mdftau['group'] == 'APP+/Tau-'], dodge=False, 
                   order = plot_order, palette = ['#52607f'], ax = ax, alpha = 0.8)
j = sns.swarmplot('structure', 'plaque density', data = mdftau[mdftau['group'] == 'APP+/Tau+'], 
                  order = plot_order, hue = 'group', s=15, ax = ax, palette = ['#C267FF'], alpha = 0.8)
plt.xticks(rotation = 90)
plt.title('Percent Plaque Coverage in Isocortex (6 months)')
ax.legend_.remove()
ax.set_ylim([0, 0.7])
ax.set_xlabel("")
ax.set_ylabel("APP/PS1")
ax.legend(loc=2, bbox_to_anchor = (0.03, 1.05))
plt.tight_layout()

plt.savefig(os.path.join(savepath, 'plaque volume in isocortex structures 6 mo APP_PS1 and Tg4510.pdf'.format(structure)), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=1000)

In [ ]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
formula = 'plaques ~ C(group) + C(structure) + C(group):C(structure)'
model = ols(formula, pltdat).fit()
aov_table = anova_lm(model, typ=2)

In [ ]:
aov_table

In [ ]:
mouse_lines = []
structures = []
ttest_results = []
pvals = []
for group in pltdat['group'].unique():
    for structure in pltdat['structure'].unique():
        ctx_distribution = pltdat[(pltdat['group'] == group) & 
                                  (pltdat['structure'] != structure)]['plaques']
        structure_distribution = pltdat[(pltdat['group'] == group) & 
                                        (pltdat['structure'] == structure)]['plaques']
        t, p = stats.ttest_ind(ctx_distribution, structure_distribution)
        structures.append(structure)
        ttest_results.append(t)
        pvals.append(p)
        mouse_lines.append(group)

results = pd.DataFrame({'Mouse Line': mouse_lines, 'Structure': structures, 't_test results': ttest_results, 
                        'fdr_corrected_pvalues': sm.stats.fdrcorrection(pvals, alpha=0.05, method='indep')[1]})

In [ ]:
results[results['fdr_corrected_pvalues'] < 0.05]

In [ ]:
dataset.head(1)

In [ ]:
alldat = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
for age in dataset['age group'].unique():
    print(age)
    for structure in iso:
        structure_acronym = ai_map[structure]
        alldat = pd.concat([alldat, get_structure_data(age, structure_acronym, 'density')])

In [ ]:
alldat['group'].unique()

In [ ]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
formula = 'plaques ~ C(group) + C(structure) + C(group):C(structure)'
for age in dataset['age group'].unique():
    agedat = alldat[alldat['age'] == age]
    model = ols(formula, agedat).fit()
    aov_table = anova_lm(model, typ=2)
    print(age)
    print(aov_table)

In [ ]:
alldat.head()

In [ ]:
ages = []
mouse_lines = []
structures = []
ttest_results = []
pvals = []
for age in alldat['age'].unique():
    for group in alldat['group'].unique():
        for structure in alldat['structure'].unique():
            ctx_distribution = alldat[(alldat['age'] == age) & 
                                      (alldat['group'] == group) & 
                                      (alldat['structure'] != structure)]['plaques']
            structure_distribution = alldat[(alldat['age'] == age) & 
                                            (alldat['group'] == group) & 
                                            (alldat['structure'] == structure)]['plaques']
            t, p = stats.ttest_ind(ctx_distribution, structure_distribution)
            if not np.isnan(p):
                ages.append(age)
                mouse_lines.append(group)
                structures.append(structure)
                ttest_results.append(t)
                pvals.append(p)

results = pd.DataFrame({'Age': ages, 'Mouse Line': mouse_lines, 'Structure': structures, 
                        't_test results': ttest_results, 
                        'fdr_corrected_pvalues': sm.stats.fdrcorrection(pvals, alpha=0.05, method='indep')[1]})

In [ ]:
sig_res = results[results['fdr_corrected_pvalues'] < 0.05]

In [ ]:
sig_res

In [ ]:
results.to_csv(os.path.join(path, 'ttest_results_isocortex.csv'))

In [ ]:
age = '18 mo'
structure = 'RSPv'
line = 'hAPP-J20'
ctx = alldat[(alldat['age'] == age) & (alldat['group'] == line) & 
                                      (alldat['structure'] != structure)]['plaques']
test = alldat[(alldat['age'] == age) & 
                                      (alldat['group'] == line) & 
                                      (alldat['structure'] == structure)]['plaques']
g = sns.distplot(ctx)
g = sns.distplot(test, bins = 10)
g.axes.xaxis.set_major_locator(ticker.MultipleLocator(0.02))
g.axes.xaxis.set_major_formatter(ticker.ScalarFormatter())
plt.title('{0} {1} {2}'.format(age, line, structure))

In [ ]:
isocortex = st.get_structures_by_acronym(['Isocortex'])[0]
hipp = st.get_structures_by_acronym(['HPF'])[0]
ptlp = st.get_structures_by_acronym(['PTLp'])[0]
hipp_ids = st.descendant_ids([hipp['id']])[0]
isohipp_ids = st.descendant_ids([isocortex['id'], hipp['id'], ptlp['id']])
print(len(isohipp_ids))
isohipp_ids = [y for x in isohipp_ids for y in x]
print(len(isohipp_ids))

In [ ]:
ia_map['FC']

In [ ]:
hipp_ss = [ids for ids in hipp_ids if ids in summary_structure_ids]
hipp_ss.remove(ia_map['FC'])
hipp_ss.remove(ia_map['IG'])
print(len(hipp_ss))

In [ ]:
alldat = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
for age in dataset['age group'].unique():
    print(age)
    for structure in hipp_ss:
        structure_acronym = ai_map[structure]
        alldat = pd.concat([alldat, get_structure_data(age, structure_acronym, 'density')])

formula = 'plaques ~ C(group) + C(structure) + C(group):C(structure)'
for age in dataset['age group'].unique():
    agedat = alldat[alldat['age'] == age]
    model = ols(formula, agedat).fit()
    aov_table = anova_lm(model, typ=2)
    print(age)
    print(aov_table)

ages = []
mouse_lines = []
structures = []
ttest_results = []
pvals = []
for age in alldat['age'].unique():
    for group in alldat['group'].unique():
        for structure in alldat['structure'].unique():
            ctx_distribution = alldat[(alldat['age'] == age) & 
                                      (alldat['group'] == group) & 
                                      (alldat['structure'] != structure)]['plaques']
            structure_distribution = alldat[(alldat['age'] == age) & 
                                            (alldat['group'] == group) & 
                                            (alldat['structure'] == structure)]['plaques']
            t, p = stats.ttest_ind(ctx_distribution, structure_distribution)
            if not np.isnan(p):
                ages.append(age)
                mouse_lines.append(group)
                structures.append(structure)
                ttest_results.append(t)
                pvals.append(p)

results = pd.DataFrame({'Age': ages, 'Mouse Line': mouse_lines, 'Structure': structures, 
                        't_test results': ttest_results, 
                        'fdr_corrected_pvalues': sm.stats.fdrcorrection(pvals, alpha=0.05, method='indep')[1]})

In [ ]:
results[results['fdr_corrected_pvalues'] < 0.05]

In [ ]:
pltdat = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
for structure in hipp_ss:
    structure_acronym = ai_map[structure]
    pltdat = pd.concat([pltdat, get_structure_data('18 mo', structure_acronym, 'density')])
mdf = pd.melt(pltdat, id_vars = ['age', 'structure', 'group', 'image_series_id'], value_name = 'plaque density')
mdf['plaque density'] = [density*100 for density in mdf['plaque density']]

fig, ax = plt.subplots(figsize = (16, 5))
g = sns.pointplot('structure', 'plaque density', hue = 'group', 
                   data = mdf[mdf['group'] == 'APP/PS1'], dodge=False, 
                   palette = mouse_line_colors,
                  hue_order = ['APP/PS1', 'hAPP-J20', 'Tg2576'], ax = ax)
#plt.xticks(rotation = 90)
plt.title('Plaque Volume in Hippocampal Structures (18 months)')
ax2 = plt.twinx()
h = sns.pointplot('structure', 'plaque density', hue = 'group', 
                   data = mdf[mdf['group'].isin(['Tg2576', 'hAPP-J20'])], 
                   palette = mouse_line_colors, dodge = False, 
                  hue_order = ['APP/PS1', 'hAPP-J20', 'Tg2576'], ax = ax2)
plt.legend(loc=1, bbox_to_anchor = (0.95, 1))
ax.legend_.remove()
#ax.set_ylim([0, 3])
ax.set_xlabel("")
ax.set_ylabel("APP/PS1")
ax2.legend_.remove()
ax2.set_ylabel("hAPP-J20, Tg2576")
#ax2.set_ylim([0, 1.5])
ax.legend(loc=2, bbox_to_anchor = (0.01, 1))
plt.tight_layout()

plt.savefig(os.path.join(savepath, 'plaque volume in hippocampal structures 18 mo.pdf'.format(structure)), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=1000)

In [ ]:
pltdat = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
for structure in hipp_ss:
    structure_acronym = ai_map[structure]
    pltdat = pd.concat([pltdat, get_structure_data('18 mo', structure_acronym, 'density')])
mdf = pd.melt(pltdat, id_vars = ['age', 'structure', 'group', 'image_series_id'], value_name = 'plaque density')
mdf['plaque density'] = [density*100 for density in mdf['plaque density']]

fig, ax = plt.subplots(figsize = (16, 6))
g = sns.boxplot('structure', 'plaque density', hue = 'group', 
                   data = mdf[mdf['group'] == 'APP/PS1'],
                   palette = mouse_line_colors, width = 0.5,
                  hue_order = ['APP/PS1', 'hAPP-J20', 'Tg2576'], ax = ax)
#plt.xticks(rotation = 90)
plt.title('Plaque Volume in Hippocampal Structures')
ax2 = plt.twinx()
h = sns.boxplot('structure', 'plaque density', hue = 'group', 
                   data = mdf[mdf['group'].isin(['Tg2576', 'hAPP-J20'])], 
                   palette = mouse_line_colors, width = 0.5,
                  hue_order = ['APP/PS1', 'hAPP-J20', 'Tg2576'], ax = ax2)
plt.legend(loc=1, bbox_to_anchor = (0.95, 1))
ax.legend_.remove()
ax.set_ylim([0, 3])
ax.set_xlabel("")
ax.set_ylabel("APP/PS1")
ax2.set_ylabel("hAPP-J20, Tg2576")
ax2.set_ylim([0, 1.5])
plt.tight_layout()

plt.savefig(os.path.join(savepath, 'plaque volume in hippocampal structures 18 mo boxplot.pdf'.format(structure)), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=1000)

In [ ]:
pltdat = pd.DataFrame(columns={'age', 'group', 'plaque density', 'structure'})
for structure in hipp_ss:
    structure_acronym = ai_map[structure]
    pltdat = pd.concat([pltdat, get_structure_data('18 mo', structure_acronym, 'density')])
mdf = pd.melt(pltdat, id_vars = ['age', 'structure', 'group', 'image_series_id'], value_name = 'plaque density')
mdf['plaque density'] = [density*100 for density in mdf['plaque density']]
g = sns.factorplot('structure', 'plaque density', hue = 'group', 
                   data = mdf[mdf['group'].isin(['APP/PS1', 'Tg2576', 'hAPP-J20'])], size=8, aspect=16 / 8, 
                   legend=False, palette = mouse_line_colors,
                  hue_order = ['APP/PS1', 'hAPP-J20', 'Tg2576'])
g.set(yscale = 'log')
g.set_ylabels('Log Plaque Density (%V)')
plt.title('Plaque Density in Hippocampal Structures')
plt.xticks(rotation = 90)
plt.legend(loc=4)
plt.tight_layout()
plt.savefig(os.path.join(savepath, 'plaque density in hippocampus structures 18 mo log.pdf'.format(structure)), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=1000)

In [ ]:
pltdat = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
for structure in hipp_ss:
    structure_acronym = ai_map[structure]
    pltdat = pd.concat([pltdat, get_structure_data('12 mo', structure_acronym, 'density')])
mdf = pd.melt(pltdat, id_vars = ['age', 'structure', 'group', 'image_series_id'], value_name = 'plaque density')
mdf['plaque density'] = [density*100 for density in mdf['plaque density']]

fig, ax = plt.subplots(figsize = (16, 5))
g = sns.pointplot('structure', 'plaque density', hue = 'group', 
                   data = mdf[mdf['group'] == 'APP/PS1'], dodge=False, 
                   palette = mouse_line_colors,
                  hue_order = ['APP/PS1', 'hAPP-J20', 'Tg2576'], ax = ax)
#plt.xticks(rotation = 90)
plt.title('Percent Plaque Coverage in Hippocampus (12 months)')
ax2 = plt.twinx()
h = sns.pointplot('structure', 'plaque density', hue = 'group', 
                   data = mdf[mdf['group'].isin(['Tg2576', 'hAPP-J20'])], 
                   palette = mouse_line_colors, dodge = False, 
                  hue_order = ['APP/PS1', 'hAPP-J20', 'Tg2576'], ax = ax2)
ax.legend_.remove()
ax.set_xlabel("")
ax.set_ylabel("APP/PS1")
ax2.legend_.remove()
ax2.set_ylabel("hAPP-J20, Tg2576")
ax2.set_ylim([0, 0.3])
ax.legend(loc = 2, bbox_to_anchor = (0.02, 1))
plt.tight_layout()

plt.savefig(os.path.join(savepath, 'plaque volume in hippocampal structures 12 mo.pdf'.format(structure)), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=1000)

In [ ]:
pltdat = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
for structure in hipp_ss:
    structure_acronym = ai_map[structure]
    pltdat = pd.concat([pltdat, get_structure_data('6 mo', structure_acronym, 'density')])
mdf = pd.melt(pltdat, id_vars = ['age', 'structure', 'group', 'image_series_id'], value_name = 'plaque density')
mdf['plaque density'] = [density*100 for density in mdf['plaque density']]

fig, ax = plt.subplots(figsize = (16, 6))
g = sns.pointplot('structure', 'plaque density', hue = 'group', 
                   data = mdf[mdf['group'].isin(['APP/PS1', 'APP/PS1/rTg4510'])], dodge=False, 
                   palette = ['#8da0cb'], ax = ax)

pltdattau = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
for structure in hipp_ss:
    structure_acronym = ai_map[structure]
    pltdattau = pd.concat([pltdattau, get_structure_data_tau('6 mo', structure_acronym, 'density')])
pltdattau = pltdattau[~pltdattau['image_series_id'].isin([682203974, 689724497, 693151433,
                                                         701806685, 715059578])] #Poor registration

mdftau = pd.melt(pltdattau, id_vars = ['age', 'structure', 'group', 'image_series_id'], 
                 value_name = 'plaque density')
mdftau['plaque density'] = [density*100 for density in mdftau['plaque density']]
h = sns.pointplot('structure', 'plaque density', hue = 'group', 
                   data = mdftau[mdftau['group'] == 'APP+/Tau-'], dodge=False, 
                   palette = ['#52607f'], ax = ax, alpha = 0.8)
j = sns.swarmplot('structure', 'plaque density', data = mdftau[mdftau['group'] == 'APP+/Tau+'], 
                  hue = 'group', s=15, ax = ax, palette = ['#C267FF'], alpha = 0.8)
plt.title('Percent Plaque Coverage in Hippocampus (6 months)')
ax.legend_.remove()
#ax.set_ylim([0, 0.7])
ax.set_xlabel("")
ax.set_ylabel("APP/PS1")
ax.legend(loc=2, bbox_to_anchor = (0.03, 1.05))
plt.tight_layout()

plt.savefig(os.path.join(savepath, 'plaque volume in hippocampus structures 6 mo APP_PS1 and Tg4510.pdf'.format(structure)), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=1000)

In [ ]:
pltdat = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
for structure in hipp_ss:
    structure_acronym = ai_map[structure]
    pltdat = pd.concat([pltdat, get_structure_data('12 mo', structure_acronym, 'density')])
mdf = pd.melt(pltdat, id_vars = ['age', 'structure', 'group', 'image_series_id'], value_name = 'plaque density')
mdf['plaque density'] = [density*100 for density in mdf['plaque density']]

fig, ax = plt.subplots(figsize = (16, 6))
g = sns.pointplot('structure', 'plaque density', hue = 'group', 
                   data = mdf[mdf['group'].isin(['APP/PS1', 'APP/PS1/rTg4510'])], dodge=False, 
                   palette = ['#8da0cb'], ax = ax)

pltdattau = pd.DataFrame(columns={'age', 'group', 'plaques', 'structure'})
for structure in hipp_ss:
    structure_acronym = ai_map[structure]
    pltdattau = pd.concat([pltdattau, get_structure_data_tau('12 mo', structure_acronym, 'density')])
pltdattau = pltdattau[~pltdattau['image_series_id'].isin([682203974, 689724497, 693151433,
                                                         701806685, 715059578])] #Poor registration

mdftau = pd.melt(pltdattau, id_vars = ['age', 'structure', 'group', 'image_series_id'], 
                 value_name = 'plaque density')
mdftau['plaque density'] = [density*100 for density in mdftau['plaque density']]
h = sns.pointplot('structure', 'plaque density', hue = 'group', 
                   data = mdftau[mdftau['group'] == 'APP+/Tau-'], dodge=False, 
                   palette = ['#52607f'], ax = ax)
j = sns.swarmplot('structure', 'plaque density', data = mdftau[mdftau['group'] == 'APP+/Tau+'], 
                  hue = 'group', s=15, ax = ax, palette = ['#C267FF'], alpha = 0.8)
plt.title('Percent Plaque Coverage in Hippocampus (12 months)')
ax.legend_.remove()
#ax.set_ylim([0, 0.7])
ax.set_xlabel("")
ax.set_ylabel("APP/PS1")
ax.legend(loc=2)
plt.tight_layout()

plt.savefig(os.path.join(savepath, 'plaque volume in hippocampus structures 12 mo APP_PS1 and Tg4510.pdf'.format(structure)), 
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=1000)

In [ ]:
def get_relative_structure_data_ctx(age, structure):
    structure_id = ia_map[structure]
    groups = []
    projection_volume = []
    for group in dataset[dataset['age group'] == age]['Mouse Line'].unique():
        isids = dataset[(dataset['Mouse Line'] == group) & (dataset['age group'] == age)]['image_series_id'].values
        for isid in isids:
            groups.append(group)
            structure_projection_volume = dat[(dat['structure_id'] == structure_id) & 
                                         (dat['image_series_id'] == isid)]['plaque_density'].values[0]
            iso_volume = dat[(dat['structure_id'] == 315) & # isocortex
                               (dat['image_series_id'] == isid)]['plaque_density'].values[0]
            hpf_volume = dat[(dat['structure_id'] == 1089) & # hippocampal formation
                               (dat['image_series_id'] == isid)]['plaque_density'].values[0]
            total_volume = iso_volume+hpf_volume
            projection_volume.append(structure_projection_volume/total_volume)
    structuredat = pd.DataFrame({'age': age, 'structure': structure, 
                                 'group': groups, 'projection volume': projection_volume})
    return structuredat

In [ ]:
olf = structure_tree.get_structures_by_acronym(['OLF'])[0]
wm = structure_tree.get_structures_by_acronym(['fiber tracts'])[0]
olf_ids = structure_tree.descendant_ids([olf['id']])[0]
print(len(olf_ids))
wm_id = wm['id']
print(wm_id)

In [ ]:
def get_mean_value_per_structure(group, age, structure_ids):
    means = []
    isids = dataset[(dataset['Mouse Line'] == group) & (dataset['age group'] == age)]['image_series_id'].values
    for structure_id in structure_ids:
        str_mean = np.mean(dat[(dat['structure_id'] == structure_id) & 
                                 (dat['image_series_id'].isin(isids))]
                             ['plaque_density'])
        means.append(str_mean*100)
    structuredat = dict(zip(structure_ids, means))
    return structuredat, len(isids)

In [ ]:
all_structure_vals, n = get_mean_value_per_structure('Tg2576', '12 mo', dat.structure_id.unique())

In [ ]:
from matplotlib import cm
structure_rgb_vals = all_structure_vals.copy()
for key in structure_rgb_vals:
    structure_rgb_vals[key] = tuple([255*i for i in cm.hot(all_structure_vals[key]*500)[:3]])
structure_rgb_vals[0] = [0.0, 0.0, 0.0]

In [ ]:
reference_space =  mcc.get_reference_space()
slice_image = reference_space.get_slice_image(1, 5000, structure_rgb_vals) # this method wants an axis and a position in microns
plt.imshow(slice_image)

In [ ]:
app_ps1_structure_vals = get_mean_value_per_structure('APP/PS1', '12 mo', dat.structure_id.unique())
app_ps1_rgb_vals = app_ps1_structure_vals.copy()
for key in app_ps1_structure_vals:
    app_ps1_rgb_vals[key] = tuple([255*i for i in cm.hot(app_ps1_structure_vals[key])[:3]])
app_ps1_rgb_vals[0] = [0, 0, 0]

w=10
h=10
image = [0,0,0,0,0,0,0,0,0,0,0,0]
index = [7000, 5000, 5000]
image[0] = reference_space.get_slice_image(0, index[0], app_ps1_rgb_vals)
image[1] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2], app_ps1_rgb_vals)), 0)
image[2] = np.rot90(reference_space.get_slice_image(1, index[1], app_ps1_rgb_vals))  
fig = plt.figure(figsize=(15, 15), facecolor='k')
columns = 3
rows = 4
for i in range(columns):
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i], cmap = cm.hot)
    plt.axis('off')

j20_structure_vals = get_mean_value_per_structure('hAPP-J20', '12 mo', dat.structure_id.unique())
j20_rgb_vals = j20_structure_vals.copy()
for key in j20_structure_vals:
    j20_rgb_vals[key] = tuple([255*i for i in cm.hot(j20_structure_vals[key]*20)[:3]])
j20_rgb_vals[0] = [0, 0, 0]

image[3] = reference_space.get_slice_image(0, index[0], j20_rgb_vals)
image[4] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2], j20_rgb_vals)), 0)
image[5] = np.rot90(reference_space.get_slice_image(1, index[1], j20_rgb_vals)) 

for i in range(columns+3):
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i], cmap = cm.hot)
    plt.axis('off')
    
tg2576_structure_vals = get_mean_value_per_structure('Tg2576', '12 mo', dat.structure_id.unique())
tg2576_rgb_vals = tg2576_structure_vals.copy()
for key in tg2576_structure_vals:
    tg2576_rgb_vals[key] = tuple([255*i for i in cm.hot(tg2576_structure_vals[key]*20)[:3]])
tg2576_rgb_vals[0] = [0, 0, 0]

image[6] = reference_space.get_slice_image(0, index[0], tg2576_rgb_vals)
image[7] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2], tg2576_rgb_vals)), 0)
image[8] = np.rot90(reference_space.get_slice_image(1, index[1], tg2576_rgb_vals)) 

for i in range(columns+6):
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i], cmap = cm.hot)
    plt.axis('off')
    
rTg4510_structure_vals = get_mean_value_per_structure('APP/PS1/rTg4510', '12 mo', dat.structure_id.unique())
rTg4510_rgb_vals = rTg4510_structure_vals.copy()
for key in rTg4510_structure_vals:
    rTg4510_rgb_vals[key] = tuple([255*i for i in cm.hot(rTg4510_structure_vals[key])[:3]])
rTg4510_rgb_vals[0] = [0, 0, 0]

image[9] = reference_space.get_slice_image(0, index[0], rTg4510_rgb_vals)
image[10] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2], rTg4510_rgb_vals)), 0)
image[11] = np.rot90(reference_space.get_slice_image(1, index[1], rTg4510_rgb_vals)) 

for i in range(columns+9):
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i], cmap = cm.hot)
    plt.axis('off')
    
fig.subplots_adjust(top=1, bottom=0.05)
cbar_ax = fig.add_subplot(70, 1, 70)
cbar = fig.colorbar(f, cax=cbar_ax, orientation='horizontal')
maxval = max(app_ps1_rgb_vals.iteritems(), key=operator.itemgetter(1))[1]
cbar_ax.set_xticklabels([0, np.round(maxval*.12, 2), np.round(maxval*.23, 2), np.round(maxval*.34, 2), 
                         np.round(maxval*.45, 2), np.round(maxval*.56, 2), np.round(maxval*.67, 2),
                        np.round(maxval*.78, 2), np.round(maxval*.89, 2), np.round(maxval, 2)])
plt.setp(plt.getp(cbar_ax, 'xticklabels'), color='w')
#plt.show()

In [ ]:
structure_vals, n = get_mean_value_per_structure('APP/PS1', '12 mo', dat.structure_id.unique())
rgb_vals = structure_vals.copy()
scale = 0.7
for key in structure_vals:
    rgb_vals[key] = tuple([255*i for i in cm.hot(structure_vals[key]*scale)[:3]])
rgb_vals[0] = [0, 0, 0]

w=10
h=10
image = [0,0,0]
index = [8000, 4500, 5000]
image[0] = reference_space.get_slice_image(0, index[0], rgb_vals)
image[1] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2], rgb_vals)), 0)
image[2] = np.rot90(reference_space.get_slice_image(1, index[1], rgb_vals))  
fig = plt.figure(figsize=(15, 15), facecolor=rgb_vals[0])
columns = 3
rows = 1
for i in range(columns*rows):
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i], cmap = cm.hot)
    plt.axis('off')
cbar = fig.colorbar(f, fraction=0.046)
cbar.set_label('Plaque % Volume', rotation=90, color = 'w')
plt.setp(plt.getp(cbar.ax.axes, 'yticklabels'), color='w')

maxval = max(structure_vals.iteritems(), key=operator.itemgetter(1))[1]/scale
cbar.ax.set_yticklabels([0, np.round(maxval*.12, 1), np.round(maxval*.23, 1), 
                         np.round(maxval*.34, 1), np.round(maxval*.45, 1), 
                         np.round(maxval*.56, 1), np.round(maxval*.67, 1),
                        np.round(maxval*.78, 1), np.round(maxval*.89, 1), np.round(maxval)])
print(n)
plt.savefig(os.path.join(savepath, 'APP_PS1 plaque map 12 mo.png'), facecolor=fig.get_facecolor(),
            bbox_inches='tight', pad_inches=0.3, format='png', dpi=500)

In [ ]:
structure_vals, n = get_mean_value_per_structure('hAPP-J20', '18 mo', dat.structure_id.unique())
rgb_vals = structure_vals.copy()
scale = 2.5
for key in structure_vals:
    rgb_vals[key] = tuple([255*i for i in cm.hot(structure_vals[key]*scale)[:3]])
rgb_vals[0] = [0, 0, 0]

w=10
h=10
image = [0,0,0]
index = [8000, 4500, 5000]
image[0] = reference_space.get_slice_image(0, index[0], rgb_vals)
image[1] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2], rgb_vals)), 0)
image[2] = np.rot90(reference_space.get_slice_image(1, index[1], rgb_vals))  
fig = plt.figure(figsize=(15, 15), facecolor=rgb_vals[0])
columns = 3
rows = 1
for i in range(columns*rows):
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i], cmap = cm.hot)
    plt.axis('off')
cbar = fig.colorbar(f, fraction=0.046)
cbar.set_label('Plaque % Volume', rotation=90, color = 'w')
plt.setp(plt.getp(cbar.ax.axes, 'yticklabels'), color='w')

maxval = max(structure_vals.iteritems(), key=operator.itemgetter(1))[1]/scale
cbar.ax.set_yticklabels([0, np.round(maxval*.12, 1), np.round(maxval*.23, 1), 
                         np.round(maxval*.34, 1), np.round(maxval*.45, 1), 
                         np.round(maxval*.56, 1), np.round(maxval*.67, 1),
                        np.round(maxval*.78, 1), np.round(maxval*.89, 1), np.round(maxval)])
print(n)
plt.savefig(os.path.join(savepath, 'hAPP-J20 plaque map 18 mo.png'), facecolor=fig.get_facecolor(),
            bbox_inches='tight', pad_inches=0.3, format='png', dpi=1000)

In [ ]:
structure_vals, n = get_mean_value_per_structure('hAPP-J20', '12 mo', dat.structure_id.unique())
rgb_vals = structure_vals.copy()
scale = 5
for key in structure_vals:
    rgb_vals[key] = tuple([255*i for i in cm.hot(structure_vals[key]*scale)[:3]])
rgb_vals[0] = [0, 0, 0]

w=10
h=10
image = [0,0,0]
index = [8000, 4500, 5000]
image[0] = reference_space.get_slice_image(0, index[0], rgb_vals)
image[1] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2], rgb_vals)), 0)
image[2] = np.rot90(reference_space.get_slice_image(1, index[1], rgb_vals))  
fig = plt.figure(figsize=(15, 15), facecolor=rgb_vals[0])
columns = 3
rows = 1
for i in range(columns*rows):
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i], cmap = cm.hot)
    plt.axis('off')
cbar = fig.colorbar(f, fraction=0.046)
cbar.set_label('Plaque % Volume', rotation=90, color = 'w')
plt.setp(plt.getp(cbar.ax.axes, 'yticklabels'), color='w')

maxval = max(structure_vals.iteritems(), key=operator.itemgetter(1))[1]/scale
cbar.ax.set_yticklabels([0, np.round(maxval*.12, 2), np.round(maxval*.23, 2), 
                         np.round(maxval*.34, 2), np.round(maxval*.45, 2), 
                         np.round(maxval*.56, 2), np.round(maxval*.67, 2),
                        np.round(maxval*.78, 2), np.round(maxval*.89, 2), np.round(maxval)])
print(n)
plt.savefig(os.path.join(savepath, 'hAPP-J20 plaque map 12 mo.png'), facecolor=fig.get_facecolor(),
            bbox_inches='tight', pad_inches=0.3, format='png', dpi=500)

In [ ]:
structure_vals, n = get_mean_value_per_structure('Tg2576', '12 mo', dat.structure_id.unique())
rgb_vals = structure_vals.copy()
scale = 10
for key in structure_vals:
    rgb_vals[key] = tuple([255*i for i in cm.hot(structure_vals[key]*scale)[:3]])
rgb_vals[0] = [0, 0, 0]

w=10
h=10
image = [0,0,0]
index = [8000, 4500, 5000]
image[0] = reference_space.get_slice_image(0, index[0], rgb_vals)
image[1] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2], rgb_vals)), 0)
image[2] = np.rot90(reference_space.get_slice_image(1, index[1], rgb_vals))  
fig = plt.figure(figsize=(15, 15), facecolor=rgb_vals[0])
columns = 3
rows = 1
for i in range(columns*rows):
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i], cmap = cm.hot)
    plt.axis('off')
cbar = fig.colorbar(f, fraction=0.046)
cbar.set_label('Plaque % Volume', rotation=90, color = 'w')
plt.setp(plt.getp(cbar.ax.axes, 'yticklabels'), color='w')

maxval = max(structure_vals.iteritems(), key=operator.itemgetter(1))[1]/scale
cbar.ax.set_yticklabels([0, np.round(maxval*.12, 2), np.round(maxval*.23, 2), 
                         np.round(maxval*.34, 2), np.round(maxval*.45, 2), 
                         np.round(maxval*.56, 2), np.round(maxval*.67, 2),
                        np.round(maxval*.78, 2), np.round(maxval*.89, 2), np.round(maxval)])
print(n)
plt.savefig(os.path.join(savepath, 'Tg2576 plaque map 12 mo.png'), facecolor=fig.get_facecolor(),
            bbox_inches='tight', pad_inches=0.3, format='png', dpi=1000)

In [ ]:
structure_vals, n = get_mean_value_per_structure('Tg2576', '18 mo', dat.structure_id.unique())
rgb_vals = structure_vals.copy()
scale = 2
for key in structure_vals:
    rgb_vals[key] = tuple([255*i for i in cm.hot(structure_vals[key]*scale)[:3]])
rgb_vals[0] = [0, 0, 0]

w=10
h=10
image = [0,0,0]
index = [8000, 4500, 5000]
image[0] = reference_space.get_slice_image(0, index[0], rgb_vals)
image[1] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2], rgb_vals)), 0)
image[2] = np.rot90(reference_space.get_slice_image(1, index[1], rgb_vals))  
fig = plt.figure(figsize=(15, 15), facecolor= rgb_vals[0])
columns = 3
rows = 1
for i in range(columns*rows):
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i], cmap = cm.hot)
    plt.axis('off')

cbar = fig.colorbar(f, orientation='vertical', 
                    fraction=0.046)
cbar_ax = fig.axes[-1]
cbar.solids.set_rasterized(True)
cbar.solids.set_edgecolor("face")
cbar.draw_all()

cbar.set_label('Plaque % Volume', rotation=90, color = 'w')
plt.setp(plt.getp(cbar.ax.axes, 'yticklabels'), color='w')

maxval = max(structure_vals.iteritems(), key=operator.itemgetter(1))[1]/scale
cbar.ax.set_yticklabels([0, np.round(maxval*.12, 1), np.round(maxval*.23, 1), 
                         np.round(maxval*.34, 1), np.round(maxval*.45, 1), 
                         np.round(maxval*.56, 1), np.round(maxval*.67, 1),
                        np.round(maxval*.78, 1), np.round(maxval*.89, 1), np.round(maxval)])
print(n)
plt.savefig(os.path.join(savepath, 'Tg2576 plaque map 18 mo.png'), facecolor=fig.get_facecolor(),
            bbox_inches='tight', pad_inches=0.3, format='png', dpi=500)

In [ ]:
structure_vals, n = get_mean_value_per_structure('APP/PS1/rTg4510', '6 mo', dat.structure_id.unique())
rgb_vals = structure_vals.copy()
scale = 0.7
for key in structure_vals:
    rgb_vals[key] = tuple([255*i for i in cm.hot(structure_vals[key]*scale)[:3]])
rgb_vals[0] = [0, 0, 0]

w=10
h=10
image = [0,0,0]
index = [8000, 4500, 5000]
image[0] = reference_space.get_slice_image(0, index[0], rgb_vals)
image[1] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2], rgb_vals)), 0)
image[2] = np.rot90(reference_space.get_slice_image(1, index[1], rgb_vals))  
fig = plt.figure(figsize=(15, 15), facecolor= rgb_vals[0])
columns = 3
rows = 1
for i in range(columns*rows):
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i], cmap = cm.hot)
    plt.axis('off')

cbar = fig.colorbar(f, orientation='vertical', 
                    fraction=0.046)
cbar_ax = fig.axes[-1]
cbar.solids.set_rasterized(True)
cbar.solids.set_edgecolor("face")
cbar.draw_all()

cbar.set_label('Plaque % Volume', rotation=90, color = 'w')
plt.setp(plt.getp(cbar.ax.axes, 'yticklabels'), color='w')

maxval = max(structure_vals.iteritems(), key=operator.itemgetter(1))[1]/scale
cbar.ax.set_yticklabels([0, np.round(maxval*.12, 1), np.round(maxval*.23, 1), 
                         np.round(maxval*.34, 1), np.round(maxval*.45, 1), 
                         np.round(maxval*.56, 1), np.round(maxval*.67, 1),
                        np.round(maxval*.78, 1), np.round(maxval*.89, 1), np.round(maxval)])
print(d)
plt.savefig(os.path.join(savepath, 'APP_PS1_rTg4510 plaque map 6 mo.png'), facecolor=fig.get_facecolor(),
            bbox_inches='tight', pad_inches=0.3, format='png', dpi=500)

In [ ]:
structure_vals, n = get_mean_value_per_structure('APP/PS1/rTg4510', '12 mo', dat.structure_id.unique())
rgb_vals = structure_vals.copy()
scale = 0.7
for key in structure_vals:
    rgb_vals[key] = tuple([255*i for i in cm.hot(structure_vals[key]*scale)[:3]])
rgb_vals[0] = [0, 0, 0]

w=10
h=10
image = [0,0,0]
index = [8000, 4500, 5000]
image[0] = reference_space.get_slice_image(0, index[0], rgb_vals)
image[1] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2], rgb_vals)), 0)
image[2] = np.rot90(reference_space.get_slice_image(1, index[1], rgb_vals))  
fig = plt.figure(figsize=(15, 15), facecolor= rgb_vals[0])
columns = 3
rows = 1
for i in range(columns*rows):
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i], cmap = cm.hot)
    plt.axis('off')

cbar = fig.colorbar(f, orientation='vertical', 
                    fraction=0.046)
cbar_ax = fig.axes[-1]
cbar.solids.set_rasterized(True)
cbar.solids.set_edgecolor("face")
cbar.draw_all()

cbar.set_label('Plaque % Volume', rotation=90, color = 'w')
plt.setp(plt.getp(cbar.ax.axes, 'yticklabels'), color='w')

maxval = max(structure_vals.iteritems(), key=operator.itemgetter(1))[1]/scale
cbar.ax.set_yticklabels([0, np.round(maxval*.12, 1), np.round(maxval*.23, 1), 
                         np.round(maxval*.34, 1), np.round(maxval*.45, 1), 
                         np.round(maxval*.56, 1), np.round(maxval*.67, 1),
                        np.round(maxval*.78, 1), np.round(maxval*.89, 1), np.round(maxval)])
print(n)
plt.savefig(os.path.join(savepath, 'APP_PS1_rTg4510 plaque map 12 mo.png'), facecolor=fig.get_facecolor(),
            bbox_inches='tight', pad_inches=0.3, format='png', dpi=500)

In [ ]:
w=10
h=10
image = [0,0,0]
index = [8000, 4500, 5000]
image[0] = reference_space.get_slice_image(0, index[0])
image[1] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2])), 0)
image[2] = np.rot90(reference_space.get_slice_image(1, index[1]))  
fig = plt.figure(figsize=(15, 15), facecolor= rgb_vals[0])
columns = 3
rows = 1
for i in range(columns*rows):
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i])
    plt.axis('off')
cbar = fig.colorbar(f, orientation='vertical', 
                    fraction=0.046)
cbar_ax = fig.axes[-1]
cbar.solids.set_rasterized(True)
cbar.solids.set_edgecolor("face")
cbar.draw_all()

cbar.set_label('Plaque % Volume', rotation=90, color = 'w')
plt.setp(plt.getp(cbar.ax.axes, 'yticklabels'), color='w')

maxval = max(structure_vals.iteritems(), key=operator.itemgetter(1))[1]/scale
cbar.ax.set_yticklabels([0, np.round(maxval*.12, 1), np.round(maxval*.23, 1), 
                         np.round(maxval*.34, 1), np.round(maxval*.45, 1), 
                         np.round(maxval*.56, 1), np.round(maxval*.67, 1),
                        np.round(maxval*.78, 1), np.round(maxval*.89, 1), np.round(maxval)])
plt.savefig(os.path.join(savepath, 'ARA with plaque map.png'), facecolor=fig.get_facecolor(),
            bbox_inches='tight', pad_inches=0.3, format='png', dpi=1000)

In [ ]:
def get_relative_mean_value_per_structure(group, age, structure_ids):
    means = []
    isids = dataset[(dataset['Mouse Line'] == group) & (dataset['age group'] == age)]['image_series_id'].values
    for structure_id in structure_ids:
        str_mean = np.mean(dat[(dat['structure_id'] == structure_id) & 
                                 (dat['image_series_id'].isin(isids))]
                             ['plaque_density'])
        root_mean = np.mean(dat[(dat['structure_id'] == 993) & 
                                 (dat['image_series_id'].isin(isids))]
                             ['plaque_density'])
        means.append(str_mean/root_mean)
    structuredat = dict(zip(structure_ids, means))
    return structuredat, len(isids)

def get_cmap(group, age, scale = 1):
    structure_vals, n = get_relative_mean_value_per_structure(group, age, dat.structure_id.unique())
    rgb_vals = structure_vals.copy()
    for key in structure_vals:
        rgb_vals[key] = tuple([255*i for i in cm.hot(structure_vals[key]*scale)[:3]])
    rgb_vals[0] = (0, 0, 0)
    return rgb_vals, n

In [ ]:
fig = plt.figure(figsize=(15, 13), facecolor='k')
columns = 3
rows = 5
image = [0,0,0,0,0,0,0,0,0,0,0,0,0,0]
index = [8000, 5000, 5000]

app_ps1_rgb_vals, app_n = get_cmap('APP/PS1', '12 mo', 0.75)
image[0] = reference_space.get_slice_image(0, index[0], app_ps1_rgb_vals)
image[1] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2], app_ps1_rgb_vals)), 0)
image[2] = np.rot90(reference_space.get_slice_image(1, index[1], app_ps1_rgb_vals))  

for i in range(columns):
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i], cmap = cm.hot)
    plt.axis('off')
plt.gcf().axes[0].set_title('APP/PS1 (n={0})'.format(app_n), size='large', color='w')

j20_rgb_vals, j20_n = get_cmap('hAPP-J20', '12 mo', 0.2)
image[3] = reference_space.get_slice_image(0, index[0], j20_rgb_vals)
image[4] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2], j20_rgb_vals)), 0)
image[5] = np.rot90(reference_space.get_slice_image(1, index[1], j20_rgb_vals)) 

for i in range(columns+3):
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i], cmap = cm.hot)
    plt.axis('off')
plt.gcf().axes[3].set_title('hAPP-J20 n={0})'.format(j20_n), size='large', color='w')

tg2576_rgb_vals, tg2576_n = get_cmap('Tg2576', '12 mo', 0.5)
image[6] = reference_space.get_slice_image(0, index[0], tg2576_rgb_vals)
image[7] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2], tg2576_rgb_vals)), 0)
image[8] = np.rot90(reference_space.get_slice_image(1, index[1], tg2576_rgb_vals)) 

for i in range(columns+6):
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i], cmap = cm.hot)
    plt.axis('off')
plt.gcf().axes[6].set_title('Tg2576, n={0})'.format(tg2576_n), size='large', color='w')
    
rTg4510_rgb_vals, rTg_n = get_cmap('APP/PS1/rTg4510', '12 mo', 0.75)
image[9] = reference_space.get_slice_image(0, index[0], rTg4510_rgb_vals)
image[10] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2], rTg4510_rgb_vals)), 0)
image[11] = np.rot90(reference_space.get_slice_image(1, index[1], rTg4510_rgb_vals)) 

for i in range(columns+9):
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i], cmap = cm.hot)
    plt.axis('off')
plt.gcf().axes[9].set_title('APP/PS1/rTg4510 (n={0})'.format(rTg_n), size='large', color='w')
    
#fig.subplots_adjust(bottom=0.1)
cbar_ax = fig.add_subplot(30, 1, 26)
cbar = fig.colorbar(f, cax=cbar_ax, orientation='horizontal')
vals, _ = get_relative_mean_value_per_structure('APP/PS1', '12 mo', dat.structure_id.unique())
maxval = max(vals.iteritems(), key=operator.itemgetter(1))[1]
cbar_ax.set_xticklabels([0, np.round(maxval*.12, 2), np.round(maxval*.23, 2), np.round(maxval*.34, 2), 
                         np.round(maxval*.45, 2), np.round(maxval*.56, 2), np.round(maxval*.67, 2),
                        np.round(maxval*.78, 2), np.round(maxval*.89, 2), np.round(maxval, 2)])
plt.setp(plt.getp(cbar_ax, 'xticklabels'), color='w')
plt.gcf().axes[12].set_title('relative plaque density', size='large', color='w')
#plt.savefig(os.path.join(path, 'plaque maps 12 mo.pdf'), facecolor=fig.get_facecolor(),
#            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=1000)

In [ ]:
fig = plt.figure(figsize=(15, 13), facecolor='k')
columns = 3
rows = 5
image = [0,0,0,0,0,0,0,0,0,0,0,0,0,0]
index = [8000, 5000, 5000]

app_ps1_rgb_vals, app_n = get_cmap('APP/PS1', '12 mo', 0.75)
image[0] = reference_space.get_slice_image(0, index[0], app_ps1_rgb_vals)
image[1] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2], app_ps1_rgb_vals)), 0)
image[2] = np.rot90(reference_space.get_slice_image(1, index[1], app_ps1_rgb_vals))  

for i in range(columns):
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i], cmap = cm.hot)
    plt.axis('off')
plt.gcf().axes[0].set_title('APP/PS1 (n={0})'.format(app_n), size='large', color='w')

j20_rgb_vals, j20_n = get_cmap('hAPP-J20', '12 mo', 0.2)
image[3] = reference_space.get_slice_image(0, index[0], j20_rgb_vals)
image[4] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2], j20_rgb_vals)), 0)
image[5] = np.rot90(reference_space.get_slice_image(1, index[1], j20_rgb_vals)) 

for i in range(columns+3):
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i], cmap = cm.hot)
    plt.axis('off')
plt.gcf().axes[3].set_title('hAPP-J20 n={0})'.format(j20_n), size='large', color='w')

tg2576_rgb_vals, tg2576_n = get_cmap('Tg2576', '12 mo', 0.5)
image[6] = reference_space.get_slice_image(0, index[0], tg2576_rgb_vals)
image[7] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2], tg2576_rgb_vals)), 0)
image[8] = np.rot90(reference_space.get_slice_image(1, index[1], tg2576_rgb_vals)) 

for i in range(columns+6):
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i], cmap = cm.hot)
    plt.axis('off')
plt.gcf().axes[6].set_title('Tg2576, n={0})'.format(tg2576_n), size='large', color='w')
    
#fig.subplots_adjust(bottom=0.1)
cbar_ax = fig.add_subplot(20, 1, 13)
cbar = fig.colorbar(f, cax=cbar_ax, orientation='horizontal')
vals, _ = get_relative_mean_value_per_structure('APP/PS1', '12 mo', dat.structure_id.unique())
maxval = max(vals.iteritems(), key=operator.itemgetter(1))[1]
cbar_ax.set_xticklabels([0, np.round(maxval*.12, 2), np.round(maxval*.23, 2), np.round(maxval*.34, 2), 
                         np.round(maxval*.45, 2), np.round(maxval*.56, 2), np.round(maxval*.67, 2),
                        np.round(maxval*.78, 2), np.round(maxval*1.08, 2), np.round(maxval*1.2, 2)])
plt.setp(plt.getp(cbar_ax, 'xticklabels'), color='w')
plt.gcf().axes[9].set_title('Relative Plaque Density', size='large', color='w')
#plt.savefig(os.path.join(path, 'plaque maps 12 mo.pdf'), facecolor=fig.get_facecolor(),
#            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=1000)

In [ ]:
fig = plt.figure(figsize=(15, 20), facecolor='k')
columns = 3
rows = 7
image = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
index = [8000, 5000, 5000]
scale=0.75

rgb_vals, n = get_cmap('APP/PS1', '4 mo', scale=scale)
image[0] = reference_space.get_slice_image(0, index[0], rgb_vals)
image[1] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2], rgb_vals)), 0)
image[2] = np.rot90(reference_space.get_slice_image(1, index[1], rgb_vals))  

for i in range(columns):
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i], cmap = cm.hot)
    plt.axis('off')
plt.gcf().axes[0].set_title('APP/PS1 4 mo (n={0})'.format(n), size='large', color='w')

rgb_vals, n = get_cmap('APP/PS1', '6 mo', scale=scale)
image[3] = reference_space.get_slice_image(0, index[0], rgb_vals)
image[4] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2], rgb_vals)), 0)
image[5] = np.rot90(reference_space.get_slice_image(1, index[1], rgb_vals)) 

for i in range(columns+3):
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i], cmap = cm.hot)
    plt.axis('off')
plt.gcf().axes[3].set_title('APP/PS1 6 mo (n={0})'.format(n), size='large', color='w')

rgb_vals, n = get_cmap('APP/PS1', '9 mo', scale=scale)
image[6] = reference_space.get_slice_image(0, index[0], rgb_vals)
image[7] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2], rgb_vals)), 0)
image[8] = np.rot90(reference_space.get_slice_image(1, index[1], rgb_vals)) 

for i in range(columns+6):
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i], cmap = cm.hot)
    plt.axis('off')
plt.gcf().axes[6].set_title('APP/PS1 9 mo (n={0})'.format(n), size='large', color='w')
    
rgb_vals, n = get_cmap('APP/PS1', '12 mo', scale=scale)
image[9] = reference_space.get_slice_image(0, index[0], rgb_vals)
image[10] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2], rgb_vals)), 0)
image[11] = np.rot90(reference_space.get_slice_image(1, index[1], rgb_vals)) 

for i in range(columns+9):
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i], cmap = cm.hot)
    plt.axis('off')
plt.gcf().axes[9].set_title('APP/PS1 12 mo (n={0})'.format(n), size='large', color='w')

rgb_vals, n = get_cmap('APP/PS1', '15 mo', scale=scale)
image[12] = reference_space.get_slice_image(0, index[0], rgb_vals)
image[13] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2], rgb_vals)), 0)
image[14] = np.rot90(reference_space.get_slice_image(1, index[1], rgb_vals)) 

for i in range(columns+12):
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i], cmap = cm.hot)
    plt.axis('off')
plt.gcf().axes[12].set_title('APP/PS1 15 mo (n={0})'.format(n), size='large', color='w')

rgb_vals, n = get_cmap('APP/PS1', '18 mo', scale=scale)
image[15] = reference_space.get_slice_image(0, index[0], rgb_vals)
image[16] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2], rgb_vals)), 0)
image[17] = np.rot90(reference_space.get_slice_image(1, index[1], rgb_vals)) 

for i in range(columns+15):
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i], cmap = cm.hot)
    plt.axis('off')
plt.gcf().axes[15].set_title('APP/PS1 18 mo (n={0})'.format(n), size='large', color='w')
    
#fig.subplots_adjust(bottom=0.1)
cbar_ax = fig.add_subplot(30, 1, 28)
cbar = fig.colorbar(f, cax=cbar_ax, orientation='horizontal')
vals, _ = get_mean_value_per_structure('APP/PS1', '12 mo', dat.structure_id.unique())
maxval = max(vals.iteritems(), key=operator.itemgetter(1))[1]
cbar_ax.set_xticklabels([0, np.round(maxval*.12, 2), np.round(maxval*.23, 2), np.round(maxval*.34, 2), 
                         np.round(maxval*.45, 2), np.round(maxval*.56, 2), np.round(maxval*.67, 2),
                        np.round(maxval*.78, 2), np.round(maxval*.89, 2), np.round(maxval, 2)])
plt.setp(plt.getp(cbar_ax, 'xticklabels'), color='w')
plt.gcf().axes[18].set_title('plaque density', size='large', color='w')
plt.savefig(os.path.join(path, 'plaque maps APP_PS1 by age.pdf'), facecolor=fig.get_facecolor(),
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=1000)

In [ ]:
fig = plt.figure(figsize=(15, 20), facecolor='k')
columns = 3
rows = 7
image = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
index = [8000, 5000, 5000]
scale = 0.2

rgb_vals, n = get_cmap('hAPP-J20', '4 mo', scale=scale)
image[0] = reference_space.get_slice_image(0, index[0], rgb_vals)
image[1] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2], rgb_vals)), 0)
image[2] = np.rot90(reference_space.get_slice_image(1, index[1], rgb_vals))  

for i in range(columns):
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i], cmap = cm.hot)
    plt.axis('off')
plt.gcf().axes[0].set_title('hAPP-J20 4 mo (n={0})'.format(n), size='large', color='w')

rgb_vals, n = get_cmap('hAPP-J20', '6 mo', scale=scale)
image[3] = reference_space.get_slice_image(0, index[0], rgb_vals)
image[4] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2], rgb_vals)), 0)
image[5] = np.rot90(reference_space.get_slice_image(1, index[1], rgb_vals)) 

for i in range(columns+3):
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i], cmap = cm.hot)
    plt.axis('off')
plt.gcf().axes[3].set_title('hAPP-J20 6 mo (n={0})'.format(n), size='large', color='w')

rgb_vals, n = get_cmap('hAPP-J20', '9 mo', scale=scale)
image[6] = reference_space.get_slice_image(0, index[0], rgb_vals)
image[7] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2], rgb_vals)), 0)
image[8] = np.rot90(reference_space.get_slice_image(1, index[1], rgb_vals)) 

for i in range(columns+6):
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i], cmap = cm.hot)
    plt.axis('off')
plt.gcf().axes[6].set_title('hAPP-J20 9 mo (n={0})'.format(n), size='large', color='w')
    
rgb_vals, n = get_cmap('hAPP-J20', '12 mo', scale=scale)
image[9] = reference_space.get_slice_image(0, index[0], rgb_vals)
image[10] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2], rgb_vals)), 0)
image[11] = np.rot90(reference_space.get_slice_image(1, index[1], rgb_vals)) 

for i in range(columns+9):
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i], cmap = cm.hot)
    plt.axis('off')
plt.gcf().axes[9].set_title('hAPP-J20 12 mo (n={0})'.format(n), size='large', color='w')

rgb_vals, n = get_cmap('hAPP-J20', '15 mo', scale=scale)
image[12] = reference_space.get_slice_image(0, index[0], rgb_vals)
image[13] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2], rgb_vals)), 0)
image[14] = np.rot90(reference_space.get_slice_image(1, index[1], rgb_vals)) 

for i in range(columns+12):
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i], cmap = cm.hot)
    plt.axis('off')
plt.gcf().axes[12].set_title('hAPP-J20 15 mo (n={0})'.format(n), size='large', color='w')

rgb_vals, n = get_cmap('hAPP-J20', '18 mo', scale=scale)
image[15] = reference_space.get_slice_image(0, index[0], rgb_vals)
image[16] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2], rgb_vals)), 0)
image[17] = np.rot90(reference_space.get_slice_image(1, index[1], rgb_vals)) 

for i in range(columns+15):
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i], cmap = cm.hot)
    plt.axis('off')
plt.gcf().axes[15].set_title('hAPP-J20 18 mo (n={0})'.format(n), size='large', color='w')
    
#fig.subplots_adjust(bottom=0.1)
cbar_ax = fig.add_subplot(30, 1, 28)
cbar = fig.colorbar(f, cax=cbar_ax, orientation='horizontal')
vals, _ = get_mean_value_per_structure('J20', '12 mo', dat.structure_id.unique())
maxval = max(vals.iteritems(), key=operator.itemgetter(1))[1]
cbar_ax.set_xticklabels([0, np.round(maxval*.12, 2), np.round(maxval*.23, 2), np.round(maxval*.34, 2), 
                         np.round(maxval*.45, 2), np.round(maxval*.56, 2), np.round(maxval*.67, 2),
                        np.round(maxval*.78, 2), np.round(maxval*.89, 2), np.round(maxval, 2)])
plt.setp(plt.getp(cbar_ax, 'xticklabels'), color='w')
plt.gcf().axes[18].set_title('relative plaque density', size='large', color='w')
plt.savefig(os.path.join(path, 'plaque maps J20 by age.pdf'), facecolor=fig.get_facecolor(),
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=1000)

In [ ]:
fig = plt.figure(figsize=(15, 20), facecolor='k')
columns = 3
rows = 4
image = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
index = [8000, 5000, 5000]
scale = 2

rgb_vals, n = get_cmap('Tg2576', '9 mo', scale=scale)
image[0] = reference_space.get_slice_image(0, index[0], rgb_vals)
image[1] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2], rgb_vals)), 0)
image[2] = np.rot90(reference_space.get_slice_image(1, index[1], rgb_vals)) 

for i in range(columns):
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i], cmap = cm.hot)
    plt.axis('off')
plt.gcf().axes[0].set_title('Tg2576 9 mo (n={0})'.format(n), size='large', color='w')
    
rgb_vals, n = get_cmap('Tg2576', '12 mo', scale=scale)
image[3] = reference_space.get_slice_image(0, index[0], rgb_vals)
image[4] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2], rgb_vals)), 0)
image[5] = np.rot90(reference_space.get_slice_image(1, index[1], rgb_vals)) 

for i in range(columns+3):
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i], cmap = cm.hot)
    plt.axis('off')
plt.gcf().axes[3].set_title('Tg2576 12 mo (n={0})'.format(n), size='large', color='w')

rgb_vals, n = get_cmap('Tg2576', '18 mo', scale=scale)
image[6] = reference_space.get_slice_image(0, index[0], rgb_vals)
image[7] = np.flip(np.rot90(reference_space.get_slice_image(2, index[2], rgb_vals)), 0)
image[8] = np.rot90(reference_space.get_slice_image(1, index[1], rgb_vals)) 

for i in range(columns+6):
    fig.add_subplot(rows, columns, i+1)
    f = plt.imshow(image[i], cmap = cm.hot)
    plt.axis('off')
plt.gcf().axes[6].set_title('Tg2576 18 mo (n={0})'.format(n), size='large', color='w')
    
#fig.subplots_adjust(bottom=0.1)
cbar_ax = fig.add_subplot(30, 1, 28)
cbar = fig.colorbar(f, cax=cbar_ax, orientation='horizontal')
vals, _ = get_mean_value_per_structure('Tg2576', '18 mo', dat.structure_id.unique())
maxval = max(vals.iteritems(), key=operator.itemgetter(1))[1]
cbar_ax.set_xticklabels([0, np.round(maxval*.12, 2), np.round(maxval*.23, 2), np.round(maxval*.34, 2), 
                         np.round(maxval*.45, 2), np.round(maxval*.56, 2), np.round(maxval*.67, 2),
                        np.round(maxval*.78, 2), np.round(maxval*.89, 2), np.round(maxval, 2)])
plt.setp(plt.getp(cbar_ax, 'xticklabels'), color='w')
plt.gcf().axes[9].set_title('relative plaque density', size='large', color='w')
plt.savefig(os.path.join(path, 'plaque maps Tg2576 by age.pdf'), facecolor=fig.get_facecolor(),
            bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=1000)

In [ ]:
from anatomy.anatomy_api import AnatomyApi
aapi = AnatomyApi()
import nrrd
c_dat = [685072427, 713927208, 714530717] # APP- mice run with plaque segmentation

c1, _ = nrrd.read(os.path.join(aapi.get_storage_directory(c_dat[0]), 
                               'grid', 'plaque', 'projection_density_25.nrrd'))

In [ ]:
np.sum(c1)

In [ ]:
c2, _ = nrrd.read(os.path.join(aapi.get_storage_directory(c_dat[1]), 
                               'grid', 'plaque', 'projection_density_25.nrrd'))
np.sum(c2)

In [ ]:
c3, _ = nrrd.read(os.path.join(aapi.get_storage_directory(c_dat[2]), 
                               'grid', 'plaque', 'projection_density_25.nrrd'))
np.sum(c3)

In [ ]:
t1, _ = nrrd.read(os.path.join(aapi.get_storage_directory(651998736), 
                               'grid', 'plaque', 'projection_density_25.nrrd'))
np.sum(t1)

In [ ]:
np.sum(t1)/np.sum(c1) #APP/PS1 18 month test/control

In [ ]:
t2, _ = nrrd.read(os.path.join(aapi.get_storage_directory(694795570), 
                               'grid', 'plaque', 'projection_density_25.nrrd'))
np.sum(t2)

In [ ]:
np.sum(t2)/np.sum(c2) #J20 18 month test/control

In [ ]:
t3, _ = nrrd.read(os.path.join(aapi.get_storage_directory(654585806), 
                               'grid', 'plaque', 'projection_density_25.nrrd'))
np.sum(t3)

In [ ]:
np.sum(t2)/np.sum(c3)